<a href="https://colab.research.google.com/github/adibayaseen/PPI-Inhibitors/blob/main/code/seqfeaturesand_gnn_generate_prediction_gnn_with_binders_and_random_both_as_negative_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set the Runtime->Change Runtime Type to GPU**

# Protein 3d structure assessment with graph neural networks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!rm -r Data
!rm -r PPI-Inhibitors
!pip install biopython
!git clone https://github.com/adibayaseen/PPI-Inhibitors
#!pip install py3Dmol

rm: cannot remove 'PPI-Inhibitors': No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:00
Cloning into 'PPI-Inhibitors'...
remote: Enumerating objects: 533, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.


In [ ]:
'''
Using Sklearn One hot encoder to encode the atoms
Output is of size N*M where N is the total number of atoms and M is the total number of encoded features
'''
import warnings
from Bio.PDB import *
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import glob
def atom1(structure):
    atomslist=np.array(sorted(np.array(['C', 'CA', 'CB', 'CG', 'CH2', 'N','NH2',  'OG','OH', 'O1', 'O2', 'SE','1']))).reshape(-1,1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(atomslist)
    atom_list=[]
    for atom in structure.get_atoms():
        if atom.get_name() in atomslist:
            atom_list.append(atom.get_name())
        else:
            atom_list.append("1")
    atoms_onehot=enc.transform(np.array(atom_list).reshape(-1,1)).toarray()
    return atoms_onehot
##############
'''
One hot encoded residue infomration using SKlearn Library

Output is N*M where N is the total number of atoms and M is the encoded features of the residues.
Any unknown  residue is mapped to 1
'''


import warnings
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import glob
from Bio.PDB import *


def res1(structure):
    residuelist=np.array(sorted(np.array(['ALA', 'ARG', 'ASN', 'ASP', 'GLN', 'GLU', 'GLY', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'CYS', 'HIS','1']))).reshape(-1,1)
    encr = OneHotEncoder(handle_unknown='ignore')
    encr.fit(residuelist)



    residue_list=[]
    for atom in structure.get_atoms():
        if atom.get_parent().get_resname() in residuelist:
            residue_list.append((atom.get_parent()).get_resname())
        else:
            residue_list.append("1")

    res_onehot=encr.transform(np.array(residue_list).reshape(-1,1)).toarray()

    return res_onehot
###########

'''
It calculates the neighbours of each atom i.e. 10 distinct neighbours
Output is  in the form of a ditionary representing an  adjacency list where each source atom and neighbouring atom is represented bby its sequence index .
'''
import warnings
from Bio.PDB import *
import numpy as np
from Bio.PDB.NeighborSearch import NeighborSearch

def neigh1(structure):
    #atom_list is a numpy array  that   contains all the atoms of the pdb file in atom object
    atom_list=np.array([atom for atom in structure.get_atoms()])

    #for atom in structure.get_atoms():
    #    atom_list.append(atom)
    #neighbour_list contains all the  neighbour atomic pairs  i.e. like if N has neighbours O and C then it is stored as [[N,C],[N,O]] i.e. has dimension N*2 where N is the total number of possible neighbours all the atoms have in an unsorted manner and it stores in the form of  atom object


    p4=NeighborSearch(atom_list)
    neighbour_list=p4.search_all(6,level="A")
    neighbour_list=np.array(neighbour_list)

    #dist is the distance between the neighbour and the source atom  i.e. dimension is N*1
    dist=np.array(neighbour_list[:,0]-neighbour_list[:,1])
    #sorting in ascending order
    place=np.argsort(dist)
    sorted_neighbour_list=neighbour_list[place]

    #old_atom_number is used for  storing atom id of the original protein before sorting
    #old_residue_number is used for storing residue number of the original protein before sorting
    source_vertex_list_atom_object=np.array(sorted_neighbour_list[:,0])
    len_source_vertex=len(source_vertex_list_atom_object)
    neighbour_vertex_with_respect_each_source_atom_object=np.array(sorted_neighbour_list[:,1])
    old_atom_number=[]
    old_residue_number=[]
    for i in atom_list:
        old_atom_number.append(i.get_serial_number())
        old_residue_number.append(i.get_parent().get_id()[1])
    old_atom_number=np.array(old_atom_number)
    old_residue_number=np.array(old_residue_number)
    req_no=len(neighbour_list)
    total_atoms=len(atom_list)
    #neigh_same_res is the 2D numpy array to store the indices of the  neighbours of  same residue and is of the shape N*10 where N is the total number of atoms
    #neigh_diff_res is 2D numpy array to store  the indices of the  neighbours of different residue
    #same_flag is used to restrict the neighbours belonging to same residue  to 10
    #diff_flag is used to restrict the neighbours belonging to different residue to 10
    neigh_same_res=np.array([[-1]*10 for i in range(total_atoms)])
    neigh_diff_res=np.array([[-1]*10 for i in range(total_atoms)])
    same_flag=[0]*total_atoms
    diff_flag=[0]*total_atoms
    for i in range(len_source_vertex):
        source_atom_id=source_vertex_list_atom_object[i].get_serial_number()
        neigh_atom_id=neighbour_vertex_with_respect_each_source_atom_object[i].get_serial_number()
        source_atom_res=source_vertex_list_atom_object[i].get_parent().get_id()[1]
        neigh_atom_res=neighbour_vertex_with_respect_each_source_atom_object[i].get_parent().get_id()[1]
        #finding out index of the source and neighbouring atoms from the original atom array with respect to their residue id and atom id
        temp_index1=np.where(source_atom_id==old_atom_number)[0]

        temp_index2=np.where(neigh_atom_id==old_atom_number)[0]
        for i1 in temp_index1:
            if old_residue_number[i1]==source_atom_res:
                source_index=i1
                break
        for i1 in temp_index2:
            if old_residue_number[i1]==neigh_atom_res:
                neigh_index=i1
                break
        #if both the residues are same

        if source_atom_res==neigh_atom_res :

            #limiting the number of neighbours of same residue to 10

            if int(same_flag[source_index])< 10:
                neigh_same_res[source_index][same_flag[source_index]]=neigh_index
                same_flag[source_index]+=1

            if int(same_flag[neigh_index])< 10:
                neigh_same_res[neigh_index][same_flag[neigh_index]]=source_index
                same_flag[neigh_index]+=1

        # if both the residues are different
        elif source_atom_res!=neigh_atom_res :

            #limiting the number of neighbours of different residues to 10

            if int(diff_flag[source_index])< 10:
                neigh_diff_res[source_index][diff_flag[source_index]]=neigh_index
                diff_flag[source_index]+=1


            if int(diff_flag[neigh_index])< 10:

                neigh_diff_res[neigh_index][diff_flag[neigh_index]]=source_index
                diff_flag[neigh_index]+=1




    return neigh_same_res,neigh_diff_res
import torch.optim as optim
import torch.nn as nn
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#from QA.data_import import get_dataloader,data1
#from QA.temp_network import GNN
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pdb
class GNN_Layer(nn.Module):

    def __init__(self, filters, v_feats, trainable=True, **kwargs):
        #pdb.set_trace()
        super(GNN_Layer, self).__init__()
        self.v_feats = v_feats
        self.filters = filters

        self.trainable = trainable
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device=device
        self.Wsv = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wdr = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wsr = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.neighbours=10
        #print("Wsv shape",self.Wsv.shape)

    def forward(self, x):
        #pdb.set_trace()
        Z,same_neigh,diff_neigh = x
        #print("input",x)
        #print(Z.shape)
        node_signals = Z@self.Wsv
        neigh_signals_same=Z@self.Wsr
        neigh_signals_diff=Z@self.Wdr
        unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(2)
        same_neigh_features=neigh_signals_same[same_neigh]*unsqueezed_same_neigh_indicator
        diff_neigh_features=neigh_signals_diff[diff_neigh]*unsqueezed_diff_neigh_indicator
        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)

        # To prevent divide by zero error
        same_norm[same_norm==0]=1
        diff_norm[diff_norm==0]=1
        neigh_same_atoms_signal = (torch.sum(same_neigh_features, axis=1))/same_norm
        neigh_diff_atoms_signal = (torch.sum(diff_neigh_features, axis=1))/diff_norm
        final_res = torch.relu(node_signals +neigh_same_atoms_signal+neigh_diff_atoms_signal)

        return final_res,same_neigh,diff_neigh

class GNN_First_Layer(nn.Module):

    def __init__(self, filters, trainable=True, **kwargs):

        super(GNN_First_Layer, self).__init__()
        self.filters = filters
        #pdb.set_trace()
        self.trainable = trainable
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device = device
        self.Wv = nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wr = nn.Parameter( torch.randn(21,self.filters, device=self.cuda_device,requires_grad=True))
        self.Wsr= nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wdr= nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.neighbours=10


    def forward(self, x):
        #pdb.set_trace()
        #print("Here i am in froward of first layer")
        atoms, residues,same_neigh,diff_neigh = x
        #print("input atoms",atoms)
        #print(atoms.shape)
        #print("Wv shape",self.Wv.shape)
        node_signals = atoms@self.Wv
        ####
        #print("input residues",residues)
        #print(residues.shape)
        #print("Wr shape",self.Wr.shape)
        ####
        residue_signals = residues@self.Wr
        #print("Wsr shape",self.Wsr.shape)
        #print("Wdr shape",self.Wdr.shape)
        neigh_signals_same=atoms@self.Wsr
        neigh_signals_diff=atoms@self.Wdr
        #print("neigh_signals_same shape",neigh_signals_same.shape)
        ####
        unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(2)
        """
        unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(1)
        unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(1)
        """
        same_neigh_features=neigh_signals_same[same_neigh]*unsqueezed_same_neigh_indicator
        diff_neigh_features=neigh_signals_diff[diff_neigh]*unsqueezed_diff_neigh_indicator
        #print("same norm",same_neigh > -1, 1)
        #1/0
        #Orignal
        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)
        same_norm = torch.sum(same_neigh > -1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1).type(torch.float)
        # To prevent divide by zero error
        same_norm[same_norm==0]=1
        diff_norm[diff_norm==0]=1
        neigh_same_atoms_signal=(torch.sum(same_neigh_features, axis=1))/same_norm
        neigh_diff_atoms_signal=(torch.sum(diff_neigh_features, axis=1))/diff_norm

        final_res = torch.relu(node_signals+residue_signals +neigh_same_atoms_signal+neigh_diff_atoms_signal)
        #pdb.set_trace()
        #print(" in First layer final_res.shape",final_res.shape,"same_neigh.shape",same_neigh.shape,"diff_neigh.shape",diff_neigh.shape)
        return final_res, same_neigh,diff_neigh


class Dense(nn.Module):
    def __init__(self, in_dims, out_dims, trainable=True, **kwargs):
        #pdb.set_trace()
        super(Dense, self).__init__()
        self.in_dims = in_dims
        self.out_dims = out_dims
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device = device

        self.W = nn.Parameter( torch.randn(self.in_dims, self.out_dims, device=self.cuda_device,requires_grad=True))

    def forward(self, x):
        #pdb.set_trace()
        Z = torch.sigmoid(torch.matmul(x, self.W))

        return Z

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        #pdb.set_trace()
        #Actual
        """
        self.conv1 = GNN_First_Layer(filters=128)
        self.conv2 = GNN_Layer(v_feats=128, filters=256)
        self.conv3 = GNN_Layer(v_feats=256, filters=512)
        self.dense = Dense(in_dims=512, out_dims=1)
        """
        #modified by me
        self.conv1 = GNN_First_Layer(filters=512)
        self.conv2 = GNN_Layer(v_feats=512, filters=1024)
        self.conv3 = GNN_Layer(v_feats=1024, filters=512)
        self.dense = Dense(in_dims=512, out_dims=1)
    def forward(self, x):
        #pdb.set_trace()
        x1=self.conv1(x)
        #pdb.set_trace()
        x2=self.conv2(x1)
        #pdb.set_trace()
        x3=self.conv3(x2)
        #pdb.set_trace()
        x=x3[0]
        x=torch.sum(x,axis=0).view(1,-1)
        #pdb.set_trace()
        x = F.normalize(x)
        """
        #pdb.set_trace()
        x5=self.dense(x)
        #pdb.set_trace()
        x6=torch.squeeze(x5,1)
        """
        #print("GNN")

        return x#x6,

    def processProtein(UniqueProtein, PdBloc):#, PdBloc):
        data_list = []
        #PdBloc='/content/PPI-Inhibitors/Data/Pdb/'
        #data=glob.glob(PdBloc+'/*')
        #data=glob.glob(PdBloc)
        #data=data[0:2]
        #assert (len(UniqueProtein) == len(data)) , "The two lists must be the same length!"
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        PData_dict={}
        for i in range(len(UniqueProtein)):
            #print('Converting PDB to Graph: {}/{}'.format(i+1, len(UniqueProtein)))
            UniqueProtein[i]=UniqueProtein[i].split('.pdb')[0]
            P1=PdBloc+UniqueProtein[i]+'.pdb'
            #if P1 in UniqueProtein:
            parser = PDBParser()
            with warnings.catch_warnings(record=True) as w:
              structure = parser.get_structure("", P1)
            one_hot_atom=(atom1(structure))
            #print("\none_hot_atom.shape",one_hot_atom.shape)
            #
            one_hot_res=(res1(structure))
            #print("\none_hot_atom.shape",one_hot_res.shape)
            neigh_same_res,neigh_diff_res=(neigh1(structure))
            #print("\neigh_same_res.shape,neigh_diff_re.shape",neigh_same_res.shape,"\n",neigh_diff_res.shape)
            # make the graph ready for PyTorch Geometrics GCN algorithms:
            one_hot_atom=torch.tensor(one_hot_atom,dtype=torch.float32).to(device)
            one_hot_res=torch.tensor(one_hot_res,dtype=torch.float32).to(device)
            neigh_same_res=torch.tensor(neigh_same_res).to(device).long()
            neigh_diff_res=torch.tensor(neigh_diff_res).to(device).long()
            GNNData = [one_hot_atom,one_hot_res,neigh_same_res,neigh_diff_res]
            #import pdb; pdb.set_trace()
            #GNNData.__setitem__('Total atoms', len(one_hot_atom))
            #data_list.append(GNNData)
            PData_dict[UniqueProtein[i]]= GNNData
        return PData_dict
def readFile(filename):
  with open(filename) as f:
    D = f.readlines()
  Name=[];PdbId=[];Ligandnames=[];SMILES=[];labels=[];
  All_data_list=[]
  from tqdm import tqdm as tqdm
  #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
  for d in tqdm(D):
      #if len(d)==6:
      name,inhibtedc,Pdbid,Ligandid,smiles,y = d.split()
      Name.append(name);PdbId.append(Pdbid);Ligandnames.append(Ligandid);SMILES.append(smiles);labels.append(float (y));
  return  PdbId,Ligandnames,SMILES,labels



In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
"""
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
"""

In [ ]:
#Compound part
!pip install kora
import kora.install.rdkit

In [ ]:
#from torch_geometric.data import InMemoryDataset, DataLoader

In [ ]:
!pip install rdkit

In [ ]:
from rdkit import Chem

In [ ]:
#from torch_geometric.nn.conv import gcn_conv
##
import glob
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
#from rdkit.Chem import MolFromSmiles
import networkx as nx
#from utils import *
# training function at each epoch
def train(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(GCN_data)))

    model.train()
    loss_fn = nn.MSELoss()
    for batch_idx, data in enumerate(train_loader):
        #data = data.to(device)
        optimizer.zero_grad()
        output,Sfeatures= model(data)#,Sfeatures
        loss = loss_fn(output, data.target.view(-1, 1).float().to(device))
        #loss = loss_fn(output, data.Target.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * len(data.x),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

def predicting(model, device,test_loader,labels):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(test_loader)))
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            #data = data.to(device)
            try:
              output,Sfeatures = model(data)#,Sfeatures
            except Exception as trr:
              print (trr)
              continue
            #output = model(data)
            #print ("output",output)
            #pdb.set_trace()
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data.target.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(),total_preds.numpy().flatten(),Sfeatures
##
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    [atom.GetIsAromatic()])

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
      c_size = mol.GetNumAtoms()
      features = []
      for atom in mol.GetAtoms() :
          feature = atom_features(atom)
          features.append( feature / sum(feature) )

      edges = []
      for bond in mol.GetBonds():
          edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
      g = nx.Graph(edges).to_directed()
      edge_index = []
      for e1, e2 in g.edges:
          edge_index.append([e1, e2])

      return c_size, features, edge_index
import os
import numpy as np
from math import sqrt
from scipy import stats
#from torch_geometric.data import InMemoryDataset, DataLoader
#from torch_geometric import data as DATA
import torch
from sklearn.metrics import roc_auc_score,roc_curve,precision_score,recall_score,average_precision_score
class PrepairDataset():
    """
    def __init__(self, root='/content/PPI-Inhibitors/Data/2p2iInhibitorsSMILES.txt', dataset='PPI-Inhibitor',
                 xd=None, xt=None, y=None, transform=None,
                 pre_transform=None,smile_graph=None,PdBloc='/content/PPI-Inhibitors/Data/Pdb/*'):

        #root is required for save preprocessed data, default is '/tmp'
        super(PrepairDataset, self).__init__(root, transform, pre_transform)
        # benchmark dataset, default = 'davis'
        self.dataset = dataset
        if os.path.isfile(self.processed_paths[0]):
            print('Pre-processed data found: {}, loading ...'.format(self.processed_paths[0]))
            self.data, self.slices = torch.load(self.processed_paths[0])
        else:
            print('Pre-processed data {} not found, doing pre-processing...'.format(self.processed_paths[0]))
            with open(root) as f:
              D = f.readlines()
            Name=[];PdbId=[];LigandId=[];SMILES=[];labels=[];
            from tqdm import tqdm as tqdm
            #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
            for d in tqdm(D):
                name,inhibtedc,Pdbid,Ligandid,smiles,y = d.split()
                Name.append(name);PdbId.append(Pdbid);LigandId.append(Ligandid);SMILES.append(smiles);labels.append(y);
            PdBloc='/content/PPI-Inhibitors/Data/Pdb'
            self.process(SMILES, PdbId, labels,smile_graph,PdBloc)
            self.data, self.slices = torch.load(self.processed_paths[0])
    """
    def ProcessfromFile(self,filename,smile_graph):
        PdBloc='/content/PPI-Inhibitors/Data/Pdb'
        with open(filename) as f:
          D = f.readlines()
        Name=[];PdbId=[];LigandId=[];SMILES=[];labels=[];
        All_data_list=[]
        from tqdm import tqdm as tqdm
        #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
        for d in tqdm(D):
            if len(d)==6:
              name,inhibtedc,Pdbid,Ligandid,smiles,y = d.split()
              Name.append(name);PdbId.append(Pdbid);LigandId.append(Ligandid);SMILES.append(smiles);labels.append(y);
            elif len(d)==2:
              Pdbid,smiles= d.split()
              PdbId.append(Pdbid);SMILES.append(smiles);labels.append(float(-1.0));
         ########
        UniqueProtein=list (set (PdbId))
        UniqueSMILES=list (set (SMILES))
        ProteinData_dict=PrepairDataset.processProtein(UniqueProtein,PdBloc)
        SmilesData_dict=PrepairDataset.processSMILES(UniqueSMILES, labels_list,smile_graph)
        for e in range(len(SMILES)):
          Pdata,Sdata,label=PdbId[e],SMILES[e],labels[e]
          #Preprocessed data
          if Sdata in SmilesData_dict and Pdata in ProteinData_dict:
            Pdata,Sdata=ProteinData_dict[Pdata],SmilesData_dict[Sdata]
            All_data=DATA.data(Pdata,Sdata,label)
            All_data_list.append(All_data)
        return All_data_list
    # Customize the process method to fit the task of drug-target affinity prediction
    # Inputs:
    # XD - list of SMILES
    #XT: target Protein PDB id
    # Y: list of labels (i.e. affinity)
    # Return: PyTorch-Geometric format processed data
    def process(xd, xt, y,smile_graph):
        assert (len(xd) == len(xt) and len(xt) == len(y)), "The three lists must be the same length!"
        data_list = []
        data_len = len(xd)
        print("data len",data_len)
        smiles_list=[];targetP_list=[];labels_list=[];
        for i in range(data_len):
            #print('Converting SMILES to graph: {}/{}'.format(i+1, data_len))
            smiles = xd[i]
            targetP = xt[i]
            labels = y[i]
            smiles_list.append(smiles);targetP_list.append(targetP);labels_list.append(labels);
        ########
        UniqueProtein=list (set (targetP_list))
        UniqueSMILES=list (set (smiles_list))
        ProteinData_dict=PrepairDataset.processProtein(UniqueProtein[0:2])#,PdBloc)
        print( "total pdbs",len(ProteinData_dict))
        SmilesData_dict=PrepairDataset.processSMILES(UniqueSMILES, labels_list,smile_graph)
        All_data_list=[]
        """
        with open(filename) as f:
          D = f.readlines()
        Name=[];PdbId=[];LigandId=[];SMILES=[];labels=[];
        All_data_list=[]
        from tqdm import tqdm as tqdm
        #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
        for d in tqdm(D):
            if len(d)==6:
              name,inhibtedc,Pdbid,Ligandid,smiles,y = d.split()
              Name.append(name);PdbId.append(Pdbid);LigandId.append(Ligandid);SMILES.append(smiles);labels.append(y);
              #Preprocessed data
              Pdata,Sdata,label=ProteinData_dict[Pdbid],SmilesData_dict[smiles],y
              All_data=DATA.data(Pdata,Sdata,label)
              All_data_list.append(All_data)
            elif len(d)==2:
              Pdbid,smiles= d.split()
              PdbId.append(Pdbid);SMILES.append(smiles);labels.append(float(-1.0));
              #Preprocessed data
              Pdata,Sdata,label=ProteinData_dict[Pdbid],SmilesData_dict[smiles],float(-1.0)
              All_data=DATA.data(Pdata,Sdata,label)
              All_data_list.append(All_data)
        return All_data_list
        """
        for e in range(len(smiles_list)):
          Pdata,Sdata,label=targetP_list[e],smiles_list[e],labels_list[e]
          #Preprocessed data
          if Sdata in SmilesData_dict and Pdata in ProteinData_dict:
            print("Sdata,Pdata,label",Sdata,Pdata,label)
            GNN_data,GCN_data=ProteinData_dict[Pdata],SmilesData_dict[Sdata]
            All_data=(GNN_data,GCN_data,label)
            All_data_list.append(All_data)
        return All_data_list
    def processProtein(UniqueProtein, PdBloc):#, PdBloc):
        data_list = []
        #PdBloc='/content/PPI-Inhibitors/Data/Pdb/'
        #data=glob.glob(PdBloc+'/*')
        #data=glob.glob(PdBloc)
        #data=data[0:2]
        #assert (len(UniqueProtein) == len(data)) , "The two lists must be the same length!"
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        PData_dict={}
        for i in range(len(UniqueProtein)):
            #print('Converting PDB to Graph: {}/{}'.format(i+1, len(UniqueProtein)))
            UniqueProtein[i]=UniqueProtein[i].split('.pdb')[0]
            P1=PdBloc+UniqueProtein[i]+'.pdb'
            #if P1 in UniqueProtein:
            parser = PDBParser()
            with warnings.catch_warnings(record=True) as w:
              structure = parser.get_structure("", P1)
            one_hot_atom=(atom1(structure))
            #print("\none_hot_atom.shape",one_hot_atom.shape)
            #
            one_hot_res=(res1(structure))
            #print("\none_hot_atom.shape",one_hot_res.shape)
            neigh_same_res,neigh_diff_res=(neigh1(structure))
            #print("\neigh_same_res.shape,neigh_diff_re.shape",neigh_same_res.shape,"\n",neigh_diff_res.shape)
            # make the graph ready for PyTorch Geometrics GCN algorithms:
            one_hot_atom=torch.tensor(one_hot_atom,dtype=torch.float32).to(device)
            one_hot_res=torch.tensor(one_hot_res,dtype=torch.float32).to(device)
            neigh_same_res=torch.tensor(neigh_same_res).to(device).long()
            neigh_diff_res=torch.tensor(neigh_diff_res).to(device).long()
            GNNData = [one_hot_atom,one_hot_res,neigh_same_res,neigh_diff_res]
            #import pdb; pdb.set_trace()
            #GNNData.__setitem__('Total atoms', len(one_hot_atom))
            #data_list.append(GNNData)
            PData_dict[UniqueProtein[i]]= GNNData
        return PData_dict
    def processSMILES(xd, targets,smile_graph):
        #assert (len(xd) == len(smile_graph)), "The two lists must be the same length!"
        data_list = []
        data_len = len(xd)
        Sdata_dict={}
        for i in range(data_len):
            print('Converting SMILES to graph: {}/{}'.format(i+1, data_len))
            smiles = xd[i]
            target=targets[i]
            print("target",target)
            mol = Chem.MolFromSmiles(smile)
            if smile in smile_graph and mol is not None and smile_graph[smiles] is not None:
              #import pdb; pdb.set_trace()
              # convert SMILES to molecular representation using rdkit
              c_size, features, edge_index = smile_graph[smiles]
              if c_size>1:
                # make the graph ready for PyTorch Geometrics GCN algorithms:
                GCNData = DATA.Data(x=torch.Tensor(features),
                                    edge_index=torch.LongTensor(edge_index).transpose(1, 0),target=torch.LongTensor([target]))
                #import pdb; pdb.set_trace()
              # GCNData.target = torch.LongTensor([target])
                #GCNData.__setitem__('c_size', torch.LongTensor([c_size]))
                # append graph, label and target sequence to data list
                #data_list.append(GCNData)
                Sdata_dict[smiles]=GCNData
        #Sdata_dict=dict(zip(xd,data_list))
        return Sdata_dict
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        print('Graph construction done. Saving to file.')
        data, slices = self.collate(data_list)
        # save preprocessed data:
        torch.save((data, slices), self.processed_paths[0])


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
def getFP(s,r = 3,nBits =2048):
    compound = Chem.MolFromSmiles(s.strip())
    if compound is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(compound, r, nBits = nBits)
        #fp = pat.GetAvalonCountFP(compound,nBits=nBits)
        m = np.zeros((0, ), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp, m)
        return m
def twomerFromSeq(s):
    k=2
    groups={'A':'1','V':'1','G':'1','I':'2','L':'2','F':'2','P':'2','Y':'3',
            'M':'3','T':'3','S':'3','H':'4','N':'4','Q':'4','W':'4',
            'R':'5','K':'5','D':'6','E':'6','C':'7'}
    crossproduct=[''.join (i) for i in product("1234567",repeat=k)]
    for i in range (0,len(crossproduct)): crossproduct[i]=int(crossproduct[i])
    ind=[]
    for i in range (0,len(crossproduct)): ind.append(i)
    combinations=dict(zip(crossproduct,ind))

    V=np.zeros(int((math.pow(7,k))))      #defines a vector of 343 length with zero entries
    try:
        for j in range (0,len(s)-k+1):
            kmer=s[j:j+k]
            c=''
            for l in range(0,k):
                c+=groups[kmer[l]]
                V[combinations[int(c)]]+=1
    except:
        count={'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0}
        for q in range(0,len(s)):
            if s[q]=='A' or s[q]=='V' or s[q]=='G':
                count['1']+=1
            if s[q]=='I' or s[q]=='L'or s[q]=='F' or s[q]=='P':
                count['2']+=1
            if s[q]=='Y' or s[q]=='M'or s[q]=='T' or s[q]=='S':
                count['3']+=1
            if s[q]=='H' or s[q]=='N'or s[q]=='Q' or s[q]=='W':
                count['4']+=1
            if s[q]=='R' or s[q]=='K':
                count['5']+=1
            if s[q]=='D' or s[q]=='E':
                count['6']+=1
            if s[q]=='C':
                count['7']+=1
        val=list(count.values()  )           #[ 0,0,0,0,0,0,0]
        key=list(count.keys()     )           #['1', '2', '3', '4', '5', '6', '7']
        m=0
        ind=0
        for t in range(0,len(val)):     #find maximum value from val
            if m<val[t]:
                m=val[t]
                ind=t
        m=key [ind]                     # m=group number of maximum occuring group alphabets in protein
        for j in range (0,len(s)-k+1):
            kmer=s[j:j+k]
            c=''
            for l in range(0,k):
                if kmer[l] not in groups:
                    c+=m
                else:
                    c+=groups[kmer[l]]
            V[combinations[int(c)]]+=1

    V=V/(len(s)-1)
    return np.array(V)
from scipy import spatial
def chainLabel(Cname_T,xl_T,Cname,xl):
    """
    Cname_T: Target chain Name
    xl_T: Target chain co-ordinates
    Cname: Off Target chain Name
    xl: Off Target chain co-ordinates
    """
    tc = getCoords(xl_T)
    nc = getCoords(xl)
    D = getDist(tc, nc, thr = 8.0)
    feats=extract_feats(generate_pair_features(D,xl_T,xl))
    return feats
def generate_pair_features(dist_info,xl,xr):
    prot_dic=make_dic()
#    pdb.set_trace()
    for rec in dist_info:

        try:
            l_letter= three_to_one(xl[rec[0]].get_resname())
            r_letter= three_to_one(xr[rec[1]].get_resname())
#            print(l_letter,l_letter)
            if (l_letter,r_letter) in prot_dic.keys():
                prot_dic[(l_letter,r_letter)]+=1
            elif (r_letter,l_letter) in prot_dic.keys():
                prot_dic[(r_letter,l_letter)]+=1
        except:
            prot_dic[('_','_')]+=1
    return prot_dic
def getCoords(R):
    """
    Get atom coordinates given a list of biopython residues
    """
    Coords = []
    for (idx, r) in enumerate(R):
        v = [ak.get_coord() for ak in r.get_list()]
        Coords.append(v)
    return Coords
def InterfaceFeatures(Complexs,pdbloc):
    Found =  listdir(pdbloc)
    InterfaceFeatures=[];InterfaceFeatures=dict(InterfaceFeatures)
    comp_id=list(set(Complexs))
    for ids in range(len(comp_id)):
        if comp_id[ids]+'.pdb' in Found:
            stx=pdbloc+'/'+comp_id[ids]+'.pdb'#'/2XA0.pdb'
            chains=Struct2chain(stx)
            for j in range(len(chains)):
                Cname_T,seq_T,L_T,xl_T=chains[j]
                for k in range(j,len(chains)):
                    Cname,seq,L,xl=chains[k]
                    #if Cname_T!=Cname and Cname!=' 'and Cname_T!=' ':
                    name=comp_id[ids]#+'_'+Cname_T+'_2_'+Cname
                    Interface=chainLabel(Cname_T,xl_T,Cname,xl)
                    InterfaceF=np.array(Interface)
                    InterfaceF=normalize(np.atleast_2d(InterfaceF), norm='l2', copy=True, axis=1, return_norm=False)
                    if name not in InterfaceFeatures.keys():
                        InterfaceFeatures[name]=Interface
    #pickle.dump(InterfaceFeatures, open(path+Filename+"_InterfaceFeatures.npy", "wb"))
    return InterfaceFeatures
def getDist(C0, C1, thr=np.inf):
    """

    """
    N0 = []
    N1 = []
    for i in range(len(C0)):
        for j in range(len(C1)):
            d = spatial.distance.cdist(C0[i], C1[j]).min()
            # dji=spatial.distance.cdist(C1[j], C0[i]).min()
            #d=min(dij,dji)
            #print d
            if (d < thr):  # and not np.isnan(self.Phi[i]) and not np.isnan(self.Phi[j])
                N0.append((i, j, d))
                N1.append((j, i, d))
    return (N0, N1)
from Bio import SeqIO
from Bio.SeqIO import FastaIO
from itertools import product
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.preprocessing import normalize
import math
import numpy as np
from Bio.Data import IUPACData
from Bio.PDB.Polypeptide import *
def prot_feats_seq(seq):
    #Interfacedict=pickle.load(open(path+"InhibitorNewModel2022/InterfaceFeatures2chainsSVM.npy","rb"))
    #InterfaceF=Interfacedict[complexname]
    aa=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    f=[]
    X = ProteinAnalysis(str(seq))
    X.molecular_weight() #throws an error if 'X' in sequence. we skip such sequences
    p=X.get_amino_acids_percent()
    dp=[]
    for a in aa:
        dp.append(p[a])
    dp=np.array(dp)
    dp=normalize(np.atleast_2d(dp), norm='l2', copy=True, axis=1, return_norm=False)
    f.extend(dp[0])

    tm=np.array(twomerFromSeq(str(seq)))
    tm=normalize(np.atleast_2d(tm), norm='l2', copy=True, axis=1,return_norm=False)
    f.extend(tm[0])
    return np.array(f)
def Struct2chain(stx):
    """
    Seq: sequence of the chain
    seq_L:sequence Length
    """
    p = PDBParser()
    L=[]
    stx=p.get_structure('X',stx)
    for model in stx:
        for C in model:
            RL=[]
            for R in C:
                RL.append(R)
            pp=PPBuilder().build_peptides(C)
            if len(pp)==0:
                pp=CaPPBuilder().build_peptides(C)
            seq=''.join([str(p.get_sequence()) for p in pp])
            #seq=''.join([p.get_sequence().tostring() for p in pp])
            seq_L=len(seq)
            L.append((C.full_id[2],seq,seq_L,RL))
    return L
def extract_feats(dic):
    feats=[]
    key_list=np.load('/content/PPI-Inhibitors/Features/'+'prote_letter_pair_keys.npy')#to keep features order same
    for key in key_list:
#        pdb.set_trace()
        feats.append(dic[(key[0].decode('utf-8'),key[1].decode('utf-8'))])

    return feats
def make_dic():
    prot_dic={}
    letters=IUPACData.protein_letters
    for i in range(len(letters)):
        for j in range(i,len(letters)):
            prot_dic[(letters[i],letters[j])]=0.0
    prot_dic[('_','_')]=0.0# for Amino acids other than 20 natural
    return prot_dic
def LoadProtein_SVM_Features(UniqueProtein,Pdbloc):
    pdbname=listdir(Pdbloc)
    InterfaceFeatures=[];InterfaceFeatures=dict(InterfaceFeatures)
    SequenceFeatures=[];SequenceFeatures=dict(SequenceFeatures)
    AllFeatures=[];AllFeatures=dict(AllFeatures)
    for  b in range(len(UniqueProtein)):
        if UniqueProtein[b]+'.pdb'in pdbname:
            stx=Pdbloc+UniqueProtein[b]+'.pdb'#directory+'/2XA0.pdb'#
            chains=Struct2chain(stx)
            #########Interface Features
            for j in range(len(chains)):
                Cname_T,seq_T,L_T,xl_T=chains[j]
                for k in range(j,len(chains)):
                    Cname,seq,L,xl=chains[k]
                    #if Cname_T!=Cname and Cname!=' 'and Cname_T!=' ':
                    name=UniqueProtein[b]#+'_'+Cname_T+'_2_'+Cname
                    Interface=chainLabel(Cname_T,xl_T,Cname,xl)
                    seq_TF=prot_feats_seq(seq_T)
                    seq_NTF=prot_feats_seq(seq)
                    SeQFeatures=(seq_TF+seq_NTF)/2
                    InterfaceF=np.array(Interface)
                    InterfaceF=normalize(np.atleast_2d(InterfaceF), norm='l2', copy=True, axis=1, return_norm=False)
                    if name not in InterfaceFeatures.keys():
                        InterfaceFeatures[name]=Interface
                        SequenceFeatures[name]=SeQFeatures
                        AllFeatures[name]=np.append(SeQFeatures,Interface)
    return InterfaceFeatures,SequenceFeatures,AllFeatures
def External_GenerateRandomNegative(posexamples):
    NegtiveRatio=1
    ###SuperDrugbank###Names
    SuperdrugNames=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="B").values#'approved_drugs_chemical_structure_identifiers.xlsx'
    SuperdrugNames=SuperdrugNames[1:]
    SuperdrugNames = np.array([s[0] for s in SuperdrugNames])
    ###############SuperDrugbank
    df_Superdrug=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="C").values
    df_Superdrug=df_Superdrug[1:]
    ###
    df_Superdrug_Compounds=np.array([c[0] for c in df_Superdrug])#3638
    SuperDrug_dict=dict (zip (SuperdrugNames,df_Superdrug_Compounds))
    ################
    #path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
    DBD5_ProteinData_dict=pickle.load(open('/content/PPI-Inhibitors/Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
    Ubench5CompNames=list (set (list (DBD5_ProteinData_dict.keys())))
    ####
    AllNeg=[];AllPos=[];complex_ligand_dict={};
    for key,val in  posexamples:
      #print(key,val,posexamples[key,val][1])
      if key not in complex_ligand_dict:
        complex_ligand_dict[key]=posexamples[key,val][1]
      else:
        #print("else",key,val)
        complex_ligand_dict[key]=np.append( complex_ligand_dict.get(key, ()) ,posexamples[key,val][1])
    Complexnames=list (complex_ligand_dict.keys())
    totalcomp=list (set (complex_ligand_dict.keys()))
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
        #print(origanlL)
        #print("complexname=",everycomp,"origanlInhibitors",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        #print(pos)
        NN =NegtiveRatio*len(pos)
        negs = []
        AllPos.extend(pos)
        while (len(negs)<NN):# and len(negs)<(len(totalligands_train)-len(origanlL)):
            LigandR = random.choice(SuperdrugNames)
            LigandR_smile=SuperDrug_dict[LigandR]
            Npair=((everycomp,LigandR_smile))
            if LigandR  not in origanlL and Npair not in AllNeg and Npair not in AllPos and getFP(LigandR_smile) is not None:
                negs.append(Npair)
#                print("Npair SuperdrugNames",Npair)
        AllNeg.extend(negs)
    #print("N=",len(AllNeg),"P",len(AllPos))
    #print("second method Cr")
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
#        print("everycomp=",everycomp,"origanlL",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        NN =NegtiveRatio*len(pos)
        negs = []
        while (len(negs)<NN):
            for everyL in origanlL:
                ComplexR = random.choice(Ubench5CompNames)
                Npair=((ComplexR,everyL))
                if ComplexR!=everycomp and Npair not in AllNeg and Npair not in AllPos:
    #                    print("ComplexR,everycomp)",Npair)
                    negs.append(Npair)
        AllNeg.extend(negs)
        ###################
    #print("N=",len(AllNeg),"P",len(AllPos))
    return np.array(AllPos),np.array(AllNeg),SuperDrug_dict

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
def PredictScorefromFile(filename,Pdbloc,Pscaler,Cscaler,trainedModel_IPPI,train_GNN,LOCOcomplexname):
  githubpath='/content/PPI-Inhibitors/'
#filename,Pdbloc,Pscaler,Cscaler,trainedModel_IPPI,train_GNN=githubpath+'Data/External data/2dyh_all.txt',githubpath+'Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model
  with open(filename) as f:
    D = f.readlines()
  InhibitedComp=[];PdbId=[];Ligandnames=[];SMILES=[];targets=[];
  All_data_list=[]
  from tqdm import tqdm as tqdm
  #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
  for d in tqdm(D):
      Pdbid,inhibtedc,Ligandid,smiles = d.split()
      if getFP(smiles) is not None:
        PdbId.append(Pdbid);Ligandnames.append(Ligandid);SMILES.append(smiles);InhibitedComp.append(inhibtedc);#labels.append(float (y));
  ################
  """
  Result_dict={}
  pos=dict (zip(zip(PdbId, Ligandnames),zip(InhibitedComp,SMILES)))
  Pos,Negs,SuperDrug_dict=External_GenerateRandomNegative(pos)
  poslabel=1.0*np.ones(len(Pos));neglabel=-1.0*np.ones(len(Negs));targets=np.append(poslabel,neglabel )
  All_examples=[];All_examples.extend(Pos);All_examples.extend(Negs)
  #Write File for External
  External_All_Examples=open('/content/drive/MyDrive/GNN-PPI-Inhibitor/_'+LOCOcomplexname+'_'+filename.split('.txt')[0].split('/')[-1]+'_External_All_Examples.txt',"w")
  """
  complexnames=[];SMILES=[];targets=[];
  #/content/PPI-Inhibitors/Data/External data/2dyh_all_External_All_Examples.txt
  with open('/content/PPI-Inhibitors/Data/External data/'+filename.split('.txt')[0].split('/')[-1]+'_External_All_Examples.txt') as f:
    D = f.readlines()
  for d in tqdm(D):
      complexname,smiles,target= d.split()
      complexnames.append(complexname);SMILES.append(smiles);targets.append(target);#All_examples.append()
  pdbname=listdir(Pdbloc);mypdb=[]
  for p in pdbname:
    if p.split('.pdb')[0] in Pdbid:
      mypdb.append(p)
  UniqueProtein=list (set (mypdb))
  External_Protein_GNN_Data_dict=PrepairDataset.processProtein(UniqueProtein,Pdbloc)
  ##########for Seq+interface features
  #pdbname=listdir(Pdbloc)
  s,i,External_ProteinSeqandInterfaceData_dict=LoadProtein_SVM_Features(UniqueProtein,Pdbloc)
  ##############3 for sequence fedatures of DBD5 pdb's
  Ubench5InterfaceandSeq_dict=pickle.load(open(githubpath+'Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
  All_External_ProteinSeqandInterfaceData_dict=dict( list (External_ProteinSeqandInterfaceData_dict.items())+list (Ubench5InterfaceandSeq_dict.items()))
  #Testing
  DBD5_Protein_GNN_Data_dict=pickle.load(open(path+'DBD5_ProteinData_dict.pickle',"rb"))
  #Pos_ProteinData_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
  All_Protein_GNN_Data_dict=dict( list (External_Protein_GNN_Data_dict.items())+list (DBD5_Protein_GNN_Data_dict.items()))
  for d in All_Protein_GNN_Data_dict:
    data=All_Protein_GNN_Data_dict[d]
    All_Protein_GNN_Data_dict[d]=[data[0].cuda(),data[1].cuda(),data[2].cuda(),data[3].cuda()]
  #####################
  Cttname=[];Ctt=[];Pttname=[];Ptt=[];
  for (complexname,ligandsmile) in zip(complexnames,SMILES):#All_examples:#
    Cttname.append(ligandsmile);Ctt.append(getFP(ligandsmile));
    Pttname.append(complexname);Ptt.append(All_External_ProteinSeqandInterfaceData_dict[complexname][0:69]);
  #standarization

  Ctt = Cscaler.transform(Ctt)
  Cttdict=dict (zip (Cttname,torch.FloatTensor( Ctt).cuda()))
  Ptt = Pscaler.transform(Ptt)
  Pttdict=dict (zip (Pttname,torch.FloatTensor( Ptt).cuda()))
  #########
  Y_t,Z,Targets=[],[],[]
  for target,(complexname,ligandsmile) in zip(targets,zip(complexnames,SMILES)):#zip(targets,All_examples):#
    #target=targets[nt]
    #IPPI_Net(All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname],Pttdict[GNNcomplex],GNN_model)
    #External_All_Examples.write(complexname+' '+ligandsmile+' '+str(target)+'\n')
    test_score=trainedModel_IPPI(All_Protein_GNN_Data_dict[complexname],Cttdict[ligandsmile],Pttdict[complexname],train_GNN)
    #print (test_score)
    test_score=test_score.cpu().data.numpy()[0]
    Z.append(test_score);Targets.append(float (target))
    #Result_dict[(complexname,Ligandname)]=test_score
  return Z,Targets


In [ ]:
!pip install torchmetrics

In [ ]:

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
class IPPI_MLP_Net(nn.Module):
    def __init__(self):
        super(IPPI_MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2560, 512)#1024)
        #self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 100)
        self.fc6 = nn.Linear(100, 1)
    def forward(self, proteinF,LigandFeatures,ProteinInterfaceF,GNN_model):
          #call GNN for protein
          #result,PFeatures=GNN_model(proteinF)
          PFeatures=GNN_model(proteinF)
          #print(PFeatures[0])
          Cfeatures=LigandFeatures#torch.FloatTensor()#.cuda()#Compound_Net(LigandFeatures)
          P_all_Features=PFeatures[0]#torch.hstack((PFeatures[0])#,ProteinInterfaceF))
          PC_Features=torch.hstack((P_all_Features,Cfeatures))
          x = torch.tanh(self.fc1(PC_Features))#.to('cuda:0')
         # x = torch.tanh(self.fc2(x))#.to('cuda:1')
          x = torch.tanh(self.fc3(x))#.to('cuda:2')
          #x = self.fc5(x)
          x = self.fc6(x)
          return x
from sklearn.metrics import roc_auc_score,roc_curve,precision_score,recall_score,average_precision_score
#For End to End LEarning
import pickle
path,githubpath='/content/drive/MyDrive/GNN-PPI-Inhibitor/','/content/PPI-Inhibitors/'
Ubench5InterfaceandSeq_dict=pickle.load(open(githubpath+'Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
Pos_seqandInterfaceF_dict=pickle.load(open(githubpath+'Features/Pos_seqandInterfaceF_dict.npy',"rb"))
Complex_AllFeatures_dict=dict( list (Pos_seqandInterfaceF_dict.items())+list (Ubench5InterfaceandSeq_dict.items()))
##############
ComplexInterfaceFeatures={}
for key in Complex_AllFeatures_dict:
  if len(key.split('_'))>1:
    compname=key.split('_')[0]
    ComplexInterfaceFeatures[compname]=Complex_AllFeatures_dict[key]#.cuda()
  else:
    ComplexInterfaceFeatures[key]=Complex_AllFeatures_dict[key]#.cuda()
#################################
CompoundFingerprintFeaturesDict=pickle.load(open(githubpath+'Features/Compound_Fingerprint_Features_Dict.npy',"rb"))
#Load Protein data for GNN
ProteinDataGNN_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
DBD5_ProteinDataGNN_dict=pickle.load(open(path+'DBD5_ProteinData_dict.pickle',"rb"))
All_ProteinData_dict=dict( list (ProteinDataGNN_dict.items())+list (DBD5_ProteinDataGNN_dict.items()))
for d in All_ProteinData_dict:
  data=All_ProteinData_dict[d]
  All_ProteinData_dict[d]=[data[0].cuda(),data[1].cuda(),data[2].cuda(),data[3].cuda()]
#########
from tqdm import tqdm as tqdm
import pickle
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
###########
#with open(githubpath+'Data/WriteAllexamplesRandomBindersIdsAll_24JAN_Binary.txt') as f:
with open(githubpath+'Data/WriteAllexamplesRandomBindersIdsAll_24JAN.txt') as f:
    D = f.readlines()
Labels=[];Ligandnames=[];Complexs=[];TestPoscomplexes=[];#SMILESlist=[];
for d in tqdm(D):
  if len(d.split())==4:
      TestPoscomp,Complexname,Ligandname,label = d.split()
  else:
      TestPoscomp,Complexname,Ligandname,label = d.split()[0],d.split()[1],(' ').join(d.split()[2:-1]),d.split()[-1]
  TestPoscomplexes.append(TestPoscomp),Ligandnames.append(Ligandname);Complexs.append(Complexname);Labels.append(float (label))
##
#########Make dictionary, Rootcomplexname=(complexname,compoundname),label
Allexamples=dict (zip(zip(TestPoscomplexes,zip(Complexs,Ligandnames)),Labels))
#Group kfold
Alldata=list (Allexamples.keys())
KK=[k[0].split('_')[0] for k in Alldata]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
from torchmetrics.classification import BinaryHingeLoss
AUC_ROC_final=[];Avg_P_final=[];Z=[];Yo=[];Y_t=[];Y_score=[];
from os import listdir
AlltestExamples=[];Externallabels=[];ExternalscoresLOCO=[];covid19_Externallabels=[];covid19_ExternalscoresLOCO=[];Y_score=[];Y_t=[];classratio_dict={};
AUC_ROC_final=[];Avg_P_final=[];
Complexs,Ligandnames, Labels=np.array(Complexs),np.array(Ligandnames),np.array(Labels)
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    Alldata=np.array(Alldata)
    train,test=Alldata[trainindex],Alldata[testindex]
    Ctr=[];Ptr=[];y_train=[];Ctrname=[];Ptrname=[];Xtr=[];G=[];Cttname=[];Ctt=[];y_test=[];Ptt=[];Pttname=[];
    #Split train and test
    for t in train:
        Ctrname.append(t[1][1]);Ctr.append(CompoundFingerprintFeaturesDict[t[1][1]]);
        #change this only for GNN Complex_AllFeatures_dict with All_ProteinData_dic and t
        #####
        GNNcomp=t[1][0].split('_')[0]#t[1][0].split('_')[0]
        SeqonlyF=ComplexInterfaceFeatures[GNNcomp][0:69]
        Ptrname.append(GNNcomp);Ptr.append(SeqonlyF);
        #print (ComplexInterfaceFeatures[GNNcomp])
        #s,i,External_ProteinSeqandInterfaceData_dict=LoadProtein_SVM_Features([GNNcomp],githubpath+'Data/Pdb/')#UniqueProtein,Pdbloc)
        #1/0
        y_train.append(Allexamples[t[0],t[1]])
    #Split train and test
    for t in test:
        GNNcomp=t[1][0].split('_')[0]
        Cttname.append(t[1][1]);Ctt.append(CompoundFingerprintFeaturesDict[t[1][1]]);
        SeqonlyF=ComplexInterfaceFeatures[GNNcomp][0:69]
        Pttname.append(GNNcomp);Ptt.append(SeqonlyF);#ComplexInterfaceFeatures[GNNcomp]);
        y_test.append(Allexamples[t[0],t[1]])
    #standarization
    Pscaler = StandardScaler().fit(Ptr)
    Cscaler = StandardScaler().fit(Ctr)
    Ctr = Cscaler.transform(Ctr)
    Ptr=Pscaler.transform(Ptr)
    Ptt=Pscaler.transform(Ptt)
    Ptrdict=dict (zip(Ptrname,torch.FloatTensor(Ptr).cuda()))
    Ctrdict=dict (zip (Ctrname,torch.FloatTensor( Ctr).cuda()))
    Ctt = Cscaler.transform(Ctt)
    Cttdict=dict (zip (Cttname,torch.FloatTensor( Ctt).cuda()))
    Pttdict=dict (zip(Pttname,torch.FloatTensor(Ptt).cuda()))
    ######################################3
    #CompoundNet=Compound_Net().cuda()7
    ################7
    IPPI_Net = IPPI_MLP_Net().cuda()
    #############
    GNN_model=GNN().cuda()
    ############
    """
    Mcomplexname,Mepoch='3TDU','10'
    IPPI_Net.load_state_dict(torch.load(path+'IPPI_Net_'+ Mcomplexname+'_Epoch'+str (Mepoch)))
    GNN_model.load_state_dict(torch.load(path+'GNN_model_'+ Mcomplexname+'_Epoch'+str (Mepoch)))
    #############
    """
    criterion = nn.MSELoss()
    #criterion = BinaryHingeLoss().cuda()
    optimizer = optim.Adam(list (IPPI_Net.parameters()) + list( GNN_model.parameters()),lr=0.0001,weight_decay=0.0)#0001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    y_train=torch.FloatTensor( y_train).cuda()
    ####
    print ("test complex ",test[0][0].split('_')[0] )
    #Done=['3UVW','2B4J','2RNY','4AJY','1YCR','4QC3','1NW9','2E3K','4GQ6','4YY6']
    Done=['4GQ6','1BXL','4ESG','2FLU','1YCQ','2XA0','3D9T','2B4J','2RNY','4AJY', '1YCR','4QC3','1BKD','1NW9','2E3K','4YY6','4GQ6','3UVW','3DAB','3WN7','1F47','3TDU']#,'1Z92'
    Batchlosslist=[]
    ################load classratio_dict based on rrot complex
    classratio_dict=pickle.load(open(githubpath+'Features/Classratio_GNNdict.npy','rb'))
    Mepoch=0
    if test[0][0].split('_')[0] in Done:
      continue
    """
    if test[0][0].split('_')[0]!=Mcomplexname:
      continue
    """
    for epoch in range(30):
      total_preds = torch.Tensor()
      total_labels = torch.Tensor()
      Batchloss=0
      epoch=epoch+int (Mepoch)
      #output=IPPI_Net(train)
      for n in range(len(train)):
        #print (train[n])
        complexname,Ligandname =train[n][1]
        GNNcomplex=complexname.split('_')[0]
        #output=IPPI_Net(GNN_model,All_ProteinData_dict[GNNcomplex],complexname,CompoundNet,CompoundFingerprintFeaturesDict[Ligandname])
        #standrized
        output=IPPI_Net(All_ProteinData_dict[GNNcomplex],Ctrdict[Ligandname],Ptrdict[GNNcomplex],GNN_model)
        #loss = criterion(output, y_train[n].view(-1))AQQAQ
        loss=criterion(output, y_train[n].reshape(1))
        #print("loss",loss.cpu().data.numpy())
        if y_train[n]==1.0:
              #print(train[n][0],classratio_dict[train[n][0]])
              loss=classratio_dict[GNNcomplex]*loss
        Batchloss=Batchloss+loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #params = list (IPPI_Net.parameters()) + list( GNN_model.parameters()) + list (CompoundNet.parameters())
        #print(train[n][1],"loss",loss.cpu().data.numpy())
      #############
      Batchlosslist.append(Batchloss.cpu().data.numpy()/len(train))
      print("Epoch",epoch,"Batch loss",Batchloss.cpu().data.numpy()/len(train))
      Y_t,Y_score=[],[]
      if epoch%5==0 or Batchloss.cpu().data.numpy()/len(train)<0.1:
        #Testing
        Y_score,Y_t=[],[]
        for nt in range(len(test)):
          complexname,Ligandname =test[nt][1]
          GNNcomplex=complexname.split('_')[0]
          test_score=IPPI_Net(All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname],Pttdict[GNNcomplex],GNN_model)
          Y_score.extend(test_score.cpu().data.numpy())
          Y_t.append(y_test[nt])
        auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
        average_PR=average_precision_score(Y_t,Y_score)
        LOCOcomplexname=test[0][0].split('_')[0]
        print("LOCOcomplexname",LOCOcomplexname,"Epoch",epoch,"auc","PR",auc_roc.round(3),average_PR.round(3),"loss",loss.cpu().data.numpy().round(3))
        file_model = path+'/IPPI_Net_'+ LOCOcomplexname+'_Epoch'+str (epoch)
        torch.save(IPPI_Net.state_dict(), file_model)
        file_model = path+'/GNN_model_'+ LOCOcomplexname+'_Epoch'+str (epoch)
        torch.save(GNN_model.state_dict(), file_model)
        #print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")#round (len(testNeg )/len(testPos),1))
        #(filename,Pdbloc,Pscaler,Cscaler,trainedModel_IPPI,train_GNN,LOCOcomplexname)
        External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model,LOCOcomplexname)
        ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
        External_Auc= roc_auc_score(External_labels, External_score)
        External_AP=average_precision_score(External_labels, External_score)
        print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
        #########\

        #covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
        Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model,LOCOcomplexname)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
        covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
        Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
        ###
        Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
        print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))
        if Batchloss.cpu().data.numpy()/len(train)<0.1:\
          break;
      ##########
    #Testing
    Y_score,Y_t=[],[]
    for nt in range(len(test)):
      complexname,Ligandname =test[nt][1]
      GNNcomplex=complexname.split('_')[0]
      test_score=IPPI_Net(All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname],Pttdict[GNNcomplex],GNN_model)
      Y_score.extend(test_score.cpu().data.numpy())
      Y_t.append(y_test[nt])
    TestComplex=test[0][0]
    Z=np.append(Z,Y_score);Yo=np.append(Yo,Y_t);
    Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
    average_P_score=average_precision_score(Y_t,Y_score)
    np.save(path+TestComplex+'_Loss.npy''_Epoch'+str (epoch),Batchlosslist)
    #np.save (path+TestComplex+'_AUC_list.npy''_Epoch'+str (epoch),AUC_list)
    #np.save (path+ TestComplex+'_PR_list.npy''_Epoch'+str (epoch),PR_list)
    ####
    file_model = path+'/IPPI_Net_'+ TestComplex+'_Epoch'+str (epoch)
    torch.save(IPPI_Net.state_dict(), file_model)
    file_model = path+'/GNN_model_'+ TestComplex+'_Epoch'+str (epoch)
    torch.save(GNN_model.state_dict(), file_model)
    Y_t=np.array(Y_t)
    #print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
    print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")#round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
    #####################External
    Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model,LOCOcomplexname)
    #Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
    #print ("RFPP",RFPP_all)
    ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
    External_Auc= roc_auc_score(External_labels, External_score)
    External_AP=average_precision_score(External_labels, External_score)
    print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
    #########\
    #covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
    Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model,LOCOcomplexname)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
    covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
    Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
    ###
    Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
    print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")
##########
External_fpr, External_tpr, External_thresholds = roc_curve(Externallabels,ExternalscoresLOCO)
External_Auc = roc_auc_score(Externallabels,ExternalscoresLOCO)
External_Auc=(External_Auc).round(2)
fig = plt.figure()
plt.plot(External_fpr, External_tpr,color='k',marker='d',label='External_Auc:{: .2f}'.format(External_Auc))
plt.title('AUCROC External');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUCROC External SVM PPI Inhibitors Random and 2 Times Binders combine Negative.pdf", bbox_inches='tight')
#######
covid19_External_fpr, covid19_External_tpr, covid19_External_thresholds = roc_curve(covid19_Externallabels,covid19_ExternalscoresLOCO)
covid19_External_Auc = roc_auc_score(covid19_Externallabels,covid19_ExternalscoresLOCO)
covid19_External_Auc=(covid19_External_Auc).round(2)
fig = plt.figure()
plt.plot(covid19_External_fpr, covid19_External_tpr,color='k',marker='d',label='covid19_External_Auc:{: .2f}'.format(covid19_External_Auc))
plt.title('covid19 AUCROC External');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUCROC covid19 External SVM PPI Inhibitors Random and 2 Times Binders combine Negative.pdf", bbox_inches='tight')
########
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr Over all complexes \n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])))#,"\n")
#######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUCROC PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
print("averge score of test",np.average(Tscores_list))
"""
np.save(path+'All_loss.npy',All_loss)
np.save (path+ 'AUC_list.npy',AUC_list)
np.save (path+ 'PR_list.npy',PR_list)
fig=plt.figure()
plt.plot(All_loss,color='k',marker=',',label='Loss')
plt.grid()
plt.legend()
fig .savefig(path+"PPI Inhibitors loss over all complexes 30 epochs.pdf", bbox_inches='tight')
############
fig=plt.figure()
plt.plot(All_loss,color='k',marker=',',label='Loss')
plt.plot( AUC_list,color='b',marker=',',label='AUC')
plt.plot( PR_list,color='m',marker=',',label='PR')
plt.grid()
plt.legend()
fig .savefig(path+"PPI Inhibitors loss-PR over all complexes.pdf", bbox_inches='tight')
"""

In [ ]:
External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model,LOCOcomplexname)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########\
#covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model,LOCOcomplexname)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))

In [ ]:
seqF,Interface,External_ProteinSeqandInterfaceData_dict=LoadProtein_SVM_Features(GNNcomp,githubpath+'Data/Pdb/')#UniqueProtein,Pdbloc)

In [ ]:
LoadProtein_SVM_Features(GNNcomp,githubpath+'Data/Pdb/')

In [ ]:
#def LoadProtein_SVM_Features(UniqueProtein,Pdbloc):
UniqueProtein,Pdbloc=[GNNcomp],githubpath+'Data/Pdb/'
pdbname=listdir(Pdbloc)
InterfaceFeatures=[];InterfaceFeatures=dict(InterfaceFeatures)
SequenceFeatures=[];SequenceFeatures=dict(SequenceFeatures)
AllFeatures=[];AllFeatures=dict(AllFeatures)
for  b in range(len(UniqueProtein)):
    if UniqueProtein[b]+'.pdb'in pdbname:
        stx=Pdbloc+UniqueProtein[b]+'.pdb'#directory+'/2XA0.pdb'#
        chains=Struct2chain(stx)
        #########Interface Features
        for j in range(len(chains)):
            Cname_T,seq_T,L_T,xl_T=chains[j]
            for k in range(j,len(chains)):
                Cname,seq,L,xl=chains[k]
                #if Cname_T!=Cname and Cname!=' 'and Cname_T!=' ':
                name=UniqueProtein[b]#+'_'+Cname_T+'_2_'+Cname
                Interface=chainLabel(Cname_T,xl_T,Cname,xl)
                seq_TF=prot_feats_seq(seq_T)
                seq_NTF=prot_feats_seq(seq)
                SeQFeatures=(seq_TF+seq_NTF)/2
                InterfaceF=np.array(Interface)
                InterfaceF=normalize(np.atleast_2d(InterfaceF), norm='l2', copy=True, axis=1, return_norm=False)
                if name not in InterfaceFeatures.keys():
                    InterfaceFeatures[name]=Interface
                    SequenceFeatures[name]=SeQFeatures
                    AllFeatures[name]=np.append(SeQFeatures,Interface)
    #return InterfaceFeatures,SequenceFeatures,AllFeatures

In [ ]:
len(SequenceFeatures['3UVW'])

In [ ]:
t.split('_')[0]

In [ ]:
ComplexInterfaceFeatures={}
for key in Complex_AllFeatures_dict:
  if len(key.split('_'))>1:
    compname=key.split('_')[0]
    ComplexInterfaceFeatures[compname]=Complex_AllFeatures_dict[key]
  else:
    ComplexInterfaceFeatures[key]=Complex_AllFeatures_dict[key]

In [ ]:
Ptrdict.keys()

In [ ]:
#####Testing
Y_score,Y_t=[],[]
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  GNNcomplex=complexname.split('_')[0]
  test_score=IPPI_Net(All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname],GNN_model)
  Y_score.extend(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])
TestComplex=test[0][0]
Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
average_P_score=average_precision_score(Y_t,Y_score)
TestComplex=test[0][0].split('_')[0]
file_model = path+'/IPPI_Net_'+ TestComplex+'_Epoch'+str (epoch)
torch.save(IPPI_Net.state_dict(), file_model)
file_model = path+'/GNN_model_'+ TestComplex+'_Epoch'+str (epoch)
torch.save(GNN_model.state_dict(), file_model)
Y_t=np.array(Y_t)
#print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")

In [ ]:
file_model

In [ ]:
#####################External
Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)
#Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
#print ("RFPP",RFPP_all)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########\
#covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")

In [ ]:
#Testing
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  GNNcomplex=complexname.split('_')[0]
  test_score=IPPI_Net(GNN_model,All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname])
  Y_score.extend(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])
TestComplex=test[0][0]
Z=np.append(Z,Y_score);Yo=np.append(Yo,Y_t);
Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
average_P_score=average_precision_score(Y_t,Y_score)
np.save(path+TestComplex+'_Loss.npy''_Epoch'+str (epoch),Batchlosslist)
#np.save (path+TestComplex+'_AUC_list.npy''_Epoch'+str (epoch),AUC_list)
#np.save (path+ TestComplex+'_PR_list.npy''_Epoch'+str (epoch),PR_list)
####
file_model = path+'/IPPI_Net_'+ TestComplex+'_Epoch'+str (epoch)
torch.save(IPPI_Net.state_dict(), file_model)
file_model = path+'/GNN_model_'+ TestComplex+'_Epoch'+str (epoch)
torch.save(GNN_model.state_dict(), file_model)
Y_t=np.array(Y_t)
#print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")#round (len(testNeg )/len(testPos),1))
AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####################External
Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)
#Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
#print ("RFPP",RFPP_all)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########\
#covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")
##########

In [ ]:
!nvidia-smi

In [ ]:
classratio_dict

In [ ]:
classratio_dict

In [ ]:
plt.plot(Batchlosslist)

In [ ]:
AUC_ROC_final=[];Avg_P_final=[];Z=[];Yo=[];Y_t=[];Y_score=[];
from os import listdir
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  GNNcomplex=complexname.split('_')[0]
  test_score=IPPI_Net(GNN_model,All_ProteinData_dict[GNNcomplex],complexname,CompoundNet,Cttdict[Ligandname])
  Y_score.extend(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])
TestComplex=test[0][0]
Z=np.append(Z,Y_score);Yo=np.append(Yo,Y_t);
Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
average_P_score=average_precision_score(Y_t,Y_score)
Y_t=np.array(Y_t)
#print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")#round (len(testNeg )/len(testPos),1))
AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####################External
Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,CompoundNet,GNN_model)
#Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
#print ("RFPP",RFPP_all)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########\
#covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,CompoundNet,GNN_model)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")

In [ ]:
loss = criterion(output, y_train[n].reshape(1)[:,]>0)

In [ ]:
loss

In [ ]:
output

In [ ]:
y_train[n].reshape(1)

In [ ]:
train[n]

In [ ]:
loss

In [ ]:
output

In [ ]:
y_train[n].view(-1)

In [ ]:
y_train[n]

In [ ]:
y_train[n-1]

In [ ]:
Batchloss=Batchloss+loss.cpu().data.numpy()
optimizer.zero_grad()
loss.backward()
optimizer.step()
#params = list (IPPI_Net.parameters()) + list( GNN_model.parameters()) + list (CompoundNet.parameters())
print("Epoch",epoch,"loss",loss.cpu().data.numpy())

In [ ]:
loss

In [ ]:
loss.data

In [ ]:
output

In [ ]:
output.shape

In [ ]:
target=y_train[n].reshape(output.shape)

In [ ]:
loss = criterion(total_preds, total_labels)

In [ ]:
loss

In [ ]:
from torchmetrics.classification import BinaryHingeLoss
bhl = BinaryHingeLoss()
loss=bhl(total_preds,total_labels)

In [ ]:
loss

In [ ]:
total_preds

In [ ]:
total_labels

In [ ]:
total_labels[:,0]>0

In [ ]:
Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)
#Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
#print ("RFPP",RFPP_all)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########\
#covid19dict,Covid19_scores,Covid19_labels,Covid19_RFPP_all=PredictRFPPfromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")

In [ ]:
Resultdict_External,Covid19_External_score,Covid19_External_labels=PredictScorefromFile(githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)

In [ ]:
Resultdict,External_score,External_labels=PredictScorefromFile(githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',IPPI_Net,GNN_model)

In [ ]:
ProteinDataGNN_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))

In [ ]:
CompoundFingerprintFeaturesDict[Ligandname]

In [ ]:
Ligandname

In [ ]:
All_ProteinData_dict[GNNcomplex]

In [ ]:
CompoundFingerprintFeaturesDict[Ligandname]

In [ ]:
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  GNNcomplex=complexname.split('_')[0]
  test_score=IPPI_Net(GNN_model,All_ProteinData_dict[GNNcomplex],complexname,CompoundFingerprintFeaturesDict[Ligandname])
  Y_score.append(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])
auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
average_PR=average_precision_score(Y_t,Y_score)
print("Epoch",epoch,"auc","PR",auc_roc.round(3),average_PR.round(3),"loss",loss.cpu().data.numpy().round(3))

In [ ]:
Covid19_RFPP_all=PredictRFPPfromFile(path+'HansonACE2hits.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)#path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Covid19_External_score,Covid19_External_labels=PredictScorefromFile(path+'HansonACE2hits.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)#,path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
###
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR,RFPP",round (Covid19_External_Auc,3), round (Covid19_External_AP,3),Covid19_RFPP_all,"\n")

In [ ]:
PredictScorefromFile(path+'HansonACE2hits.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)

In [ ]:
Externalpath='/content/PPI-Inhibitors/Data/External data/'
#Result=PredictScorefromFile(Externalpath+'External1.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
Pdbloc=Externalpath+'pdb/'
#trainedModel_IPPI,train_GNN=IPPI_Net,GNN_model
with open(Externalpath+'2dyh_all.txt') as f:
  D = f.readlines()
InhibitedComp=[];PdbId=[];Ligandnames=[];SMILES=[];labels=[];
All_data_list=[]
from tqdm import tqdm as tqdm
#2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
for d in tqdm(D):
    #if len(d)==6:
    Pdbid,inhibtedc,Ligandid,smiles = d.split()
    PdbId.append(Pdbid);Ligandnames.append(Ligandid);SMILES.append(smiles);InhibitedComp.append(inhibtedc);#labels.append(float (y));
################
pdbname=listdir(Pdbloc)
mypdb=[]
for p in pdbname:
  if p.split('.pdb')[0] in Pdbid:
    mypdb.append(p)
UniqueProtein=list (set (mypdb))
pos=dict (zip(zip(PdbId, Ligandnames),zip(InhibitedComp,SMILES)))
Pos,Negs,labels=External_GenerateRandomNegative(pos)
1/0
#Negs=External_GenerateRandomNegative(pos)
#omplexnames=list (complex_ligand_dict.keys())
AllNeg=[];AllPos=[];
totalcomp=list(set (np.array([([t][0]) for t in pos])))
External_ProteinData_dict=PrepairDataset.processProtein(UniqueProtein,Pdbloc)
Result_dict={}
#Testing
Y_t,Z=[],[]
for nt in range(len(InhibitedComp)):
  complexname,Ligandname, ligandsmile,inhibitC =PdbId[nt], Ligandnames[nt], SMILES[nt],InhibitedComp[nt]
  test_score=trainedModel_IPPI(train_GNN,External_ProteinData_dict[complexname],complexname,ligandsmile)
  if test_score!=0.0:
      test_score=test_score.cpu().data.numpy()[0]
      Z.append(test_score)
      Result_dict[(complexname,Ligandname)]=(inhibitC,test_score)

In [ ]:
    posexamples=dict (zip(zip(PdbId, Ligandnames),zip(InhibitedComp,SMILES)))
    NegtiveRatio=1
    ###SuperDrugbank
    ###Names
    SuperdrugNames=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="B").values#'approved_drugs_chemical_structure_identifiers.xlsx'
    SuperdrugNames=SuperdrugNames[1:]
    SuperdrugNames = np.array([s[0] for s in SuperdrugNames])
    ###############
    ###SuperDrugbank
    df_Superdrug=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="C").values
    df_Superdrug=df_Superdrug[1:]
    ###
    df_Superdrug_Compounds=np.array([c[0] for c in df_Superdrug])#3638
    SuperDrug_dict=dict (zip (SuperdrugNames,df_Superdrug_Compounds))
    ################
    path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
    DBD5_ProteinData_dict=pickle.load(open(path+'DBD5_ProteinData_dict.pickle',"rb"))
    Ubench5CompNames=list (set (list (DBD5_ProteinData_dict.keys())))
    ####
    AllNeg=[];AllPos=[];
    complex_ligand_dict={}
    for key,val in  posexamples:
      print(key,val,posexamples[key,val][1])
      if key not in complex_ligand_dict:
        complex_ligand_dict[key]=posexamples[key,val][1]
      else:
        #print("else",key,val)
        complex_ligand_dict[key]=np.append( complex_ligand_dict.get(key, ()) ,posexamples[key,val][1])
    Complexnames=list (complex_ligand_dict.keys())
    #totalligands_train=list (set (totalligands_train))
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
        print("complexname=",everycomp,"origanlInhibitors",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        NN =NegtiveRatio*len(pos)
        negs = []
        AllPos.extend(pos)
        while (len(negs)<NN):# and len(negs)<(len(totalligands_train)-len(origanlL)):
            LigandR = random.choice(SuperdrugNames)
            LigandR_smile=SuperDrug_dict[LigandR]
            Npair=((everycomp,LigandR_smile))
            if LigandR  not in origanlL and Npair not in AllNeg and Npair not in AllPos and getFP(LigandR_smile) is not None:
                negs.append(Npair)
#                print("Npair SuperdrugNames",Npair)
        AllNeg.extend(negs)
    print("N=",len(AllNeg),"P",len(AllPos))
    print("second method Cr")
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
#        print("everycomp=",everycomp,"origanlL",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        NN =NegtiveRatio*len(pos)
        negs = []
        while (len(negs)<NN):
            for everyL in origanlL:
                ComplexR = random.choice(Ubench5CompNames)
                Npair=((ComplexR,everyL))
                if ComplexR!=everycomp and Npair not in AllNeg and Npair not in AllPos:
    #                    print("ComplexR,everycomp)",Npair)
                    negs.append(Npair)
        AllNeg.extend(negs)
        ###################
    print("N=",len(AllNeg),"P",len(AllPos))

In [ ]:
Covid19_RFPP_all=PredictRFPPfromFile(path,path+'/HansonACE2hits.txt',path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Covid19_External_score,Covid19_External_labels=PredictScorefromFileSVM(path,path+'/HansonACE2hits.txt',path+'/External data for Inhibitors/protein pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)

In [ ]:
Externalpath='/content/PPI-Inhibitors/Data/External data/'
Resultdict,External_score,External_labels=PredictScorefromFile(Externalpath+'HansonACE2hits.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'HansonACE2hits.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
print ("RFPP",RFPP_all)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR,RFPP",round (External_Auc,3), round (External_AP,3),RFPP_all,"\n")

In [ ]:
Externalpath='/content/PPI-Inhibitors/Data/External data/'
Resultdict,Tscores,Tlabels=PredictScorefromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
Resultdict,Tscores,Tlabels,RFPP_all=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
print ("RFPP",RFPP_all)
T_Auc=roc_auc_score(np.array(Tlabels), np.array(Tscores))#Tscores,Tlabels
T_PR=average_precision_score(np.array(Tlabels), np.array(Tscores))
print (T_Auc,T_PR)
Tscores_list.append(Tscores)

In [ ]:
PredictScorefromFile(Externalpath+'External1.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)

In [ ]:
GNN_model(All_ProteinData_dict[complexname])


In [ ]:
test

In [ ]:
len(Tscores)

In [ ]:
plt.plot(Tscores,color='r')
plt.plot(Tlabels,color='b')

In [ ]:
complexname,Ligandname =test[nt]

In [ ]:
complexname

In [ ]:
All_examples[complexname,ligandname][1]

In [ ]:
All_examples

In [ ]:
Superdrug_SMILES#Features all unique Compounds of Superdrugbank and truepositive examples

In [ ]:
Actual_Compound

In [ ]:
Pi

In [ ]:
Actual_Compound

In [ ]:

##########loop
RFPP_all=[]
perntile_values = [0,1,5,10,20,40,50,60,70,80,90,95,99,100]#99
for Pi_index in range(len(P2C_dict)):
  ####For one example Pi
  Pi=P2C_dict[Pi_index ]#pick one protein P index
  Pi_Feature=External_ProteinData_dict[Pi]#P features of that index
  Actual_Compound=P2C_dict[Pi]#Actul_Compound paired with Pi
  #Actual_Compound_Features=np.array([U[c] for c in Actual_Compound])
  Ctt=np.vstack((Actual_Compound,Superdrug_SMILES))#Features all unique Compounds of Superdrugbank and truepositive examples
  Ptt=np.array([Pi for i in range(len(Ctt))])#Copy same feature of protein equal to number of unique compounds


In [ ]:
P2C_dict[Pi_index]

In [ ]:
Resultdict,Tscores,Tlabels,RFPP_list=PredictRFPPfromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)

In [ ]:
T_Auc=roc_auc_score(np.array(Tlabels), np.array(Tscores))#Tscores,Tlabels
T_PR=average_precision_score(np.array(Tlabels), np.array(Tscores))
print (T_Auc,T_PR)

In [ ]:
Tscores

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,precision_score,recall_score,average_precision_score
#For End to End LEarning
"""
path='/content/PPI-Inhibitors/'
#############
model=GNN()

model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
#########
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
"""
pos_Pid,Ligandnames,pos_smiles,pos_label=readFile('/content/PPI-Inhibitors/Data/2p2iInhibitorsSMILES.txt')
###
"""
UniqueProtein=list (set (pos_Pid))
ProteinData_dict=PrepairDataset.processProtein(UniqueProtein)##PdBloc)
path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
# save dictionary to pickle file
with open(path+'ProteinData_dict.pickle', 'wb') as filename:
    pickle.dump(ProteinData_dict, filename, protocol=pickle.HIGHEST_PROTOCOL)
"""
#pickle.dump(path+'ProteinData_dict.npy',ProteinData_dict)
path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
import pickle
ProteinData_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
import pandas as pd
fields=['(Complexname)','Binders SMILES']
df=pd.read_csv('/content/PPI-Inhibitors/Data/BindersWithComplexname.csv', skipinitialspace=True, usecols=fields)
# See the keys
#print(df.keys())
neg_Pidname,neg_smiles=df[df.keys()[0]].values,df[df.keys()[1]].values

neg_Pid=[n.split('_')[0] for n in neg_Pidname]\

New_binders_dict=AppendlistinDict(neg_Pid,neg_smiles)
####
neg_label=-1.0*(np.ones(len(neg_smiles)))
#####
Z = []; Yo = []; A = [];Yp=[]
AUC_ROC_final=[];Precision_final=[];Recall_final=[];Avg_P_final=[];
####
comp=dict(zip(zip (pos_Pid,Ligandnames),zip(pos_Pid,pos_smiles)))
#####
complex_ligand_dict=AppendlistinDict(pos_Pid,Ligandnames)
Ldict=dict (zip(Ligandnames,pos_smiles))
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
ProteinData_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[]; PR_list=[];All_loss=[]
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateNegativeBinderonly(trainindex,CC,complex_ligand_dict,New_binders_dict)
    trainSMILES=[Ldict[t[1]] for t in trainPos]
    traincomp=[t[0] for t in trainPos]
    trainPos=list (zip(traincomp,trainSMILES))
    train=np.vstack((trainPos,trainNeg))
    poslabel=1.0*np.ones(len(trainPos));neglabel=-1.0*np.ones(len(trainNeg))
    y_train=np.append(poslabel,neglabel )
    ###########
    testPos,testNeg=GenerateNegativeBinderonly(testindex,CC,complex_ligand_dict,New_binders_dict)
    testSMILES=[Ldict[t[1]] for t in testPos]
    testcomp=[t[0] for t in testPos]
    testPos=list (zip(testcomp,testSMILES))
    poslabel=1.0*np.ones(len(testPos));neglabel=-1.0*np.ones(len(testNeg))
    y_test=np.append(poslabel,neglabel )
    test=np.vstack((testPos,testNeg))
    #All examples passed for negative examples generation
    ######################################3
    IPPI_Net = IPPI_MLP_Net().cuda()
    #############
    GNN_model=GNN().cuda()
    #####
    epoch='29'
    #Done=['1BKD','3D9T','1YCQ','2FLU','4QC3','2B4J','3UVW']
    Done=['3UVW','2B4J','2RNY','4AJY','1YCR','4QC3','1NW9','2E3K','4GQ6','4YY6','1BXL','4ESG','3DAB','2FLU','1YCQ','2XA0','1Z92', '1F47','3TDU','3D9T','1BKD','3WN7']#
    complexnameT=test[0][0]
    #print ("test complex ",complexnameT)
    for d in Done:
      if complexnameT==d:
        IPPI_Net.load_state_dict(torch.load(path+'IPPI_Net_'+ complexnameT+'_Epoch'+str (epoch)))
        GNN_model.load_state_dict(torch.load(path+'GNN_model_'+ complexnameT+'_Epoch'+str (epoch)))
        #Testing
        Y_t,Y_score=[],[]
        for nt in range(len(test)):
          complexname,Ligandname =test[nt]
          #print("n",n)
          test_score=IPPI_Net(GNN_model,ProteinData_dict[complexname],complexname,Ligandname)
          if test_score!=0.0:
              Y_score.extend(test_score.cpu().data.numpy())
              Y_t.append(y_test[nt])
        Z=np.append(Z,Y_score);Yo=np.append(Yo,Y_t);
        Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
        average_P_score=average_precision_score(Y_t,Y_score)
        LP=np.load(path+complexnameT+'_Loss.npy''_Epoch'+str (epoch)+'.npy')
        AUC_epoch=np.load(path+ complexnameT+'_AUC_list.npy''_Epoch'+str (epoch)+'.npy')
        PR_epoch=np.load (path+ complexnameT+'_PR_list.npy''_Epoch'+str (epoch)+'.npy')
        All_loss.extend(LP);AUC_list.extend(AUC_epoch); PR_list.extend(PR_epoch)
        ######
        fig=plt.figure()
        plt.plot(LP,color='k',marker=',',label='Loss_'+complexnameT)
        plt.grid()
        plt.legend()
        fig .savefig(path+complexnameT+"_Loss.pdf", bbox_inches='tight')
        ########
        Y_t=np.array(Y_t)
        #print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
        print(complexnameT,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1))#,"\n")#round (len(testNeg )/len(testPos),1))
        AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
        #####External
        Externalpath='/content/PPI-Inhibitors/Data/External data/'
        #Resultdict,Tscores=PredictScorefromFile(Externalpath+'External1.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
        Resultdict,Tscores=PredictScorefromFile(Externalpath+'2dyh_all.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
        print("Tscores",Tscores)
        1/0
        #import pdb;pdb.set_trace()
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr Over all complexes \n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])))#,"\n")
#######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUCROC PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))

np.save(path+'All_loss.npy',All_loss)
np.save (path+ 'AUC_list.npy',AUC_list)
np.save (path+ 'PR_list.npy',PR_list)
####
fig=plt.figure()
plt.plot(All_loss,color='k',marker=',',label='Loss')
plt.grid()
plt.legend()
fig .savefig(path+"PPI Inhibitors loss over all complexes 30 epochs.pdf", bbox_inches='tight')
############
fig=plt.figure()
plt.plot(All_loss,color='k',marker=',',label='Loss')
plt.plot( AUC_list,color='b',marker=',',label='AUC')
plt.plot( PR_list,color='m',marker=',',label='PR')
plt.grid()
plt.legend()
fig .savefig(path+"PPI Inhibitors loss-PR over all complexes.pdf", bbox_inches='tight')

In [ ]:
Externalpath='/content/PPI-Inhibitors/Data/External data/'
Result=PredictScorefromFile(Externalpath+'External1.txt',Externalpath+'pdb/',IPPI_Net,GNN_model)
print(Result)

In [ ]:
#For End to End LEarning
"""
path='/content/PPI-Inhibitors/'
#############
model=GNN()

model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
#########
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
"""
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
#DBD5_ProteinData_dict=pickle.load(open(path+'DBD5_ProteinData_dict.pickle',"rb"))
Pos_ProteinData_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
"""
All_ProteinData_dict=dict( list (Pos_ProteinData_dict.items())+list (DBD5_ProteinData_dict.items()))
for d in All_ProteinData_dict:
  data=All_ProteinData_dict[d]
  All_ProteinData_dict[d]=[data[0].cuda(),data[1].cuda(),data[2].cuda(),data[3].cuda()]
"""
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[]; PR_list=[];
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateRandomNegative(trainindex,CC,complex_ligand_dict,Ldict)
    train=np.vstack((trainPos,trainNeg))
    train_SMILES=[Ldict[t[1]] for t in train]
    traincomp=[t[0] for t in train]
    train=list (zip(traincomp,train_SMILES))
    poslabel=1.0*np.ones(len(trainPos));neglabel=-1.0*np.ones(len(trainNeg))
    y_train=np.append(poslabel,neglabel )
    ###########
    testPos,testNeg=GenerateRandomNegative(testindex,CC,complex_ligand_dict,Ldict)
    test=np.vstack((testPos,testNeg))
    testSMILES=[Ldict[t[1]] for t in test]
    testcomp=[t[0] for t in test]
    test=list (zip(testcomp,testSMILES))
    poslabel=1.0*np.ones(len(testPos));neglabel=-1.0*np.ones(len(testNeg))
    y_test=np.append(poslabel,neglabel )
    #All examples passed for negative examples generation
    ######################################3
    IPPI_Net = IPPI_MLP_Net().cuda()
    #############
    GNN_model=GNN().cuda()
    #####
    """
    complexname,epoch='3UVW','0'
    IPPI_Net.load_state_dict(torch.load(path+'IPPI_Net_'+ complexname+'_Epoch'+str (epoch)))
    GNN_model.load_state_dict(torch.load(path+'GNN_model_'+ complexname+'_Epoch'+str (epoch)))
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(list (IPPI_Net.parameters()) + list( GNN_model.parameters()),lr=0.01,weight_decay=0.0001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    y_train=torch.FloatTensor( y_train).cuda()
    ####
    print ("test complex ",testPos[0][0],"test complex[-1]",testPos[-1][0])
    #Done=['3UVW','2B4J','2RNY','4AJY','1YCR','4QC3','1NW9','2E3K','4GQ6','4YY6']
    #3UVW 2B4J 2RNY 4QC3 1YCR 4AJY 1NW9 4GQ6 2E3K 4YY6 1BXL 4ESG 2FLU 1YCQ 3D9T 2XA0 3WN7 1Z92 3DAB 3TDU 1F47 1BKD
    Done=['4QC3','1NW9','2E3K','4GQ6','3WN7','1YCQ','2XA0','1Z92', '1F47','4ESG','1BKD','4YY6','3D9T','3DAB','1BXL','2FLU','3TDU']#'3UVW','2B4J','2RNY','4AJY','1YCR',
    if test[0][0] in Done:
      continue
    for epoch in range(100):
      total_preds = torch.Tensor()
      total_labels = torch.Tensor()
      for n in range(len(train)):
        complexname,Ligandname =train[n]
        #if complexname=='3UVW':
          #continue
        #print("n",n)
        output=IPPI_Net(GNN_model,All_ProteinData_dict,complexname,Ligandname)
        #print("output",output)
        if output!=0.0:
          total_preds = torch.cat((total_preds, output.cpu()), 0)
          total_labels = torch.cat((total_labels, y_train[n].view(-1, 1).cpu()), 0)
      #output_list=torch.FloatTensor(output_list,requires_grad=True).cuda
      #target=torch.FloatTensor(target).cuda
      loss = criterion(total_preds, total_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      params = list(IPPI_Net.parameters())
      print("Epoch",epoch,"loss",loss.cpu().data.numpy())
      #############
      Y_t,Y_score=[],[]
      if epoch%3==0:
        for nt in range(len(test)):
          complexname,Ligandname =test[nt]
          test_score=IPPI_Net(GNN_model,All_ProteinData_dict,complexname,Ligandname)
          #print("nt",nt,"test_score",test_score)
          if test_score!=0.0:
            Y_score.append(test_score.cpu().data.numpy())
            Y_t.append(y_test[nt])
        auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
        average_PR=average_precision_score(Y_t,Y_score)
        print("test comp",testPos[0][0],"Epoch",epoch,"auc","PR",auc_roc.round(3),average_PR.round(3),"loss",loss.cpu().data.numpy().round(3))
  #            print("Fauc_roc",auc_roc)
        AUC_list.append(auc_roc);PR_list.append(average_PR)
        Y_t,Y_score=[],[]
        LP.append(loss.cpu().data.numpy())
    #if (loss.cpu().data.numpy()<0.22):
      #break;
    #Testing
    for nt in range(len(test)):
      complexname,Ligandname =test[nt]
      #print("n",n)
      test_score=IPPI_Net(GNN_model,All_ProteinData_dict,complexname,Ligandname)
      if test_score!=0.0:
          Y_score.extend(test_score.cpu().data.numpy())
          Y_t.append(y_test[nt])

    print (complexname,"test complex ",testPos[0][0],testPos[-1][0])
    TestComplex=testPos[0][0]
    Z=np.append(Z,Y_score);Yo=np.append(Yo,Y_t);
    Auc=roc_auc_score(np.array(Y_t), np.array(Y_score))
    average_P_score=average_precision_score(Y_t,Y_score)
    np.save(path+TestComplex+'_Loss.npy''_Epoch'+str (epoch),LP)
    np.save (path+ TestComplex+'_AUC_list.npy''_Epoch'+str (epoch),AUC_list)
    np.save (path+ TestComplex+'_PR_list.npy''_Epoch'+str (epoch),PR_list)
    ####
    file_model = path+'/IPPI_Net_Random_'+ TestComplex+'_Epoch'+str (epoch)
    torch.save(IPPI_Net.state_dict(), file_model)
    file_model = path+'/GNN_model_Random_'+ TestComplex+'_Epoch'+str (epoch)
    torch.save(GNN_model.state_dict(), file_model)
    Y_t=np.array(Y_t)
    #print("len of Z,Yo",len(Z),len(Yo),"Test P",complexname,"Final AUC",Auc,"Final PR",average_P_score)
    print(TestComplex,"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",np.sum(Y_t[Y_t==1.0]),"\t",np.abs(np.sum(Y_t[Y_t==-1.0])),"\t",round (np.abs(np.sum(Y_t[Y_t==-1.0]))/np.sum(Y_t[Y_t==1.0]),1),"\n")#round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
#######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
plt.figure()
np.save(path+'Loss.npy',LP)
np.save (path+ 'AUC_listEpoch5.npy',AUC_list)
plt.plot(LP)
plt.plot( AUC_list)
plt.grid()

In [ ]:
Y_t=np.array(Y_t)

In [ ]:
LP

In [ ]:
 for nt in range(len(test)):
  complexname,Ligandname =test[nt]
  test_score=IPPI_Net(GNN_model,ProteinData_dict,complexname,Ligandname)
  print("nt",nt,"test_score",test_score)
  if test_score!=0.0:
    Y_score.append(test_score.cpu().data.numpy())
    Y_t.append(y_test[nt])
auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())

In [ ]:
 for nt in range(len(test)):
  complexname,Ligandname =test[nt]
  test_score=IPPI_Net(GNN_model,ProteinData_dict,complexname,Ligandname)
  print("nt",nt,"test_score",test_score)
  if output!=0.0:
    Y_score.append(test_score.cpu().data.numpy())
    Y_t.append(y_test[nt])
auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())

In [ ]:
for nt in range(len(test)):
  complexname,Ligandname =test[nt]
  print("n",n)
  test_score=IPPI_Net(GNN_model,ProteinData_dict,complexname,Ligandname)
  print("test_score",test_score)
  Y_score.append(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])
auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())

In [ ]:
test_score=IPPI_Net(GNN_model,ProteinData_dict,complexname,Ligandname)

In [ ]:
test_score

In [ ]:
Y_t.append(y_test[nt])

In [ ]:
for nt in range(len(test)):
  complexname,Ligandname =test[nt]
  #print("n",n)
  test_score=IPPI_Net(GNN_model,ProteinData_dict,complexname,Ligandname)
  Y_score.extend(test_score.cpu().data.numpy())
  Y_t.extend(y_test[nt].cpu().data.numpy())
  auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
  print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())
#            print("Fauc_roc",auc_roc)
  AUC_list.append(auc_roc)
  Y_t,Y_score=[],[]
  LP.append(loss.cpu().data.numpy())

In [ ]:
loss

In [ ]:
total_labels

In [ ]:
torch.cat(output_list,output,dim=0)

In [ ]:
output_list

In [ ]:
output_list=torch.FloatTensor(output_list).cuda
target=torch.FloatTensor(target).cuda
loss = criterion(output_list, y_train)
optimizer.zero_grad()
loss.backward()
optimizer.step()

params = list(IPPI_Net.parameters())

In [ ]:
output

In [ ]:
output_list

In [ ]:
output.cpu().detach().numpy()[0]

In [ ]:
trainPos

In [ ]:
getFP(Ligandname)

In [ ]:
output_list

In [ ]:
output_list=torch.FloatTensor(output_list).cuda

In [ ]:
loss = criterion(output_list, y_train[0:1000])

In [ ]:
 y_train=np.append(1.0*np.ones(len(trainPos)),-1.0*np.ones(len(trainNeg)))
 y_train=torch.FloatTensor( y_train).cuda()

In [ ]:
 leny_train

In [ ]:
output

In [ ]:
path='/content/PPI-Inhibitors/'
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[];
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateNegativeBinderonly(trainindex,CC,complex_ligand_dict,New_binders_dict)
    train=np.vstack((trainPos,trainNeg))
    #All examples passed for negative examples generation
#        posexamples=testindex
    testPos,testNeg=GenerateNegativeBinderonly(testindex,CC,complex_ligand_dict,New_binders_dict)

    1/0
#        AlltestExamples.extend(testPos);AlltestExamples.extend(testNeg);
    ######
    Ctr=[];Ptr=[];y_train=[]
    for t in trainPos:
        if getFP(Ldict[t[1]])is not None:
            Ctr.append(getFP(Ldict[t[1]]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(1)
    #####
    for t in trainNeg:
        if getFP(t[1]) is not None:
            Ctr.append(getFP(t[1]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(-1);
    #######
    PC_train_Features=np.hstack((Ptr,Ctr))
#        ######

    Ctt=[];Ptt=[];y_test=[]
    for t in testPos:
        if getFP(Ldict[t[1]])is not None:
            Ctt.append(getFP(Ldict[t[1]]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(1)
    ####
    for t in testNeg:
#             if t[0]!='1TUE_A_2_B':
        if getFP(t[1]) is not None:
            Ctt.append(getFP(t[1]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(-1);
    #######
    PC_test_Features=np.hstack((Ptt,Ctt))
    X_train,X_test=PC_train_Features,PC_test_Features
    ######################################3

    IPPI_Net = IPPI_MLP_Net().cuda()
    #print(IPPI_MLP_Net())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(IPPI_Net.parameters(),lr=0.0001,weight_decay=0.001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    X_train=torch.FloatTensor(X_train).cuda()
    y_train=torch.FloatTensor( y_train).cuda()
    X_test=torch.FloatTensor(X_test).cuda()
    y_test=torch.FloatTensor( y_test).cuda()
    for epoch in range(100):
      output = IPPI_Net(X_train)
      output=torch.squeeze(output, 1)
      target =   y_train
      loss = criterion(output, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      params = list(IPPI_Net.parameters())
      test_score = IPPI_Net(X_test)
      test_score=torch.squeeze(test_score, 1)
      Y_score.extend(test_score.cpu().data.numpy())
      Y_t.extend(y_test.cpu().data.numpy())
      test_score.tolist()
      y_test.tolist()
      auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())
#            print("Fauc_roc",auc_roc)
      AUC_list.append(auc_roc)
      Y_t,Y_score=[],[]
      LP.append(loss.cpu().data.numpy())
    #Testing
    test_score = IPPI_Net(X_test)
    test_score=torch.squeeze(test_score, 1)
    test_score=test_score.cpu().data.numpy()
    y_test=y_test.cpu().data.numpy()
    Z=np.append(Z,test_score);Yo=np.append(Yo,y_test);
    Auc=roc_auc_score(np.array(y_test), np.array(test_score))
    print("len of Z,Yo",len(Z),len(Yo),"Test P",t[0],"Final auc_roc",Auc)

    average_P_score=average_precision_score(y_test,test_score)
    print(testPos[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",len(testPos),"\t",len(testNeg),"\t",round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
  ######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
#    ######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
plt.figure()
plt.plot(LP)
plt.plot( AUC_list)
plt.grid()

In [ ]:
testNeg

In [ ]:
path='/content/PPI-Inhibitors/'
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[];
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateNegativeBinderonly(trainindex,CC,complex_ligand_dict,New_binders_dict)
    train=np.vstack((trainPos,trainNeg))
    #All examples passed for negative examples generation
#        posexamples=testindex
    testPos,testNeg=GenerateNegativeBinderonly(testindex,CC,complex_ligand_dict,New_binders_dict)
#        AlltestExamples.extend(testPos);AlltestExamples.extend(testNeg);
    ######
    Ctr=[];Ptr=[];y_train=[]
    for t in trainPos:
        if getFP(Ldict[t[1]])is not None:
            Ctr.append(getFP(Ldict[t[1]]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(1)
    #####
    for t in trainNeg:
        if getFP(t[1]) is not None:
            Ctr.append(getFP(t[1]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(-1);
    #######
    PC_train_Features=np.hstack((Ptr,Ctr))
#        ######

    Ctt=[];Ptt=[];y_test=[]
    for t in testPos:
        if getFP(Ldict[t[1]])is not None:
            Ctt.append(getFP(Ldict[t[1]]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(1)
    ####
    for t in testNeg:
#             if t[0]!='1TUE_A_2_B':
        if getFP(t[1]) is not None:
            Ctt.append(getFP(t[1]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(-1);
    #######
    PC_test_Features=np.hstack((Ptt,Ctt))
    X_train,X_test=PC_train_Features,PC_test_Features
    ######################################3

    IPPI_Net = IPPI_MLP_Net().cuda()
    #print(IPPI_MLP_Net())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(IPPI_Net.parameters(),lr=0.0001,weight_decay=0.001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    X_train=torch.FloatTensor(X_train).cuda()
    y_train=torch.FloatTensor( y_train).cuda()
    X_test=torch.FloatTensor(X_test).cuda()
    y_test=torch.FloatTensor( y_test).cuda()
    for epoch in range(100):
      output = IPPI_Net(X_train)
      output=torch.squeeze(output, 1)
      target =   y_train
      loss = criterion(output, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if loss<0.1:
        break;
      params = list(IPPI_Net.parameters())
      test_score = IPPI_Net(X_test)
      test_score=torch.squeeze(test_score, 1)
      Y_score.extend(test_score.cpu().data.numpy())
      Y_t.extend(y_test.cpu().data.numpy())
      test_score.tolist()
      y_test.tolist()
      auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())
#            print("Fauc_roc",auc_roc)
      AUC_list.append(auc_roc)
      Y_t,Y_score=[],[]
      LP.append(loss.cpu().data.numpy())
    #Testing
    test_score = IPPI_Net(X_test)
    test_score=torch.squeeze(test_score, 1)
    test_score=test_score.cpu().data.numpy()
    y_test=y_test.cpu().data.numpy()
    Z=np.append(Z,test_score);Yo=np.append(Yo,y_test);
    Auc=roc_auc_score(np.array(y_test), np.array(test_score))
    print("len of Z,Yo",len(Z),len(Yo),"Test P",t[0],"Final auc_roc",Auc)

    average_P_score=average_precision_score(y_test,test_score)
    print(testPos[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",len(testPos),"\t",len(testNeg),"\t",round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
  ######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
#    ######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
plt.figure()
plt.plot(LP)
plt.plot( AUC_list)
plt.grid()

In [ ]:
path='/content/PPI-Inhibitors/'
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[];
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateNegativeBinderonly(trainindex,CC,complex_ligand_dict,New_binders_dict)
    train=np.vstack((trainPos,trainNeg))
    #All examples passed for negative examples generation
#        posexamples=testindex
    testPos,testNeg=GenerateNegativeBinderonly(testindex,CC,complex_ligand_dict,New_binders_dict)
#        AlltestExamples.extend(testPos);AlltestExamples.extend(testNeg);
    ######
    Ctr=[];Ptr=[];y_train=[]
    for t in trainPos:
        if getFP(Ldict[t[1]])is not None:
            Ctr.append(getFP(Ldict[t[1]]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(1)
    #####
    for t in trainNeg:
        if getFP(t[1]) is not None:
            Ctr.append(getFP(t[1]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(-1);
    #######
    PC_train_Features=np.hstack((Ptr,Ctr))
#        ######

    Ctt=[];Ptt=[];y_test=[]
    for t in testPos:
        if getFP(Ldict[t[1]])is not None:
            Ctt.append(getFP(Ldict[t[1]]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(1)
    ####
    for t in testNeg:
#             if t[0]!='1TUE_A_2_B':
        if getFP(t[1]) is not None:
            Ctt.append(getFP(t[1]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(-1);
    #######
    PC_test_Features=np.hstack((Ptt,Ctt))
    X_train,X_test=PC_train_Features,PC_test_Features
    ######################################3
    Max_auc_roc=0.0
    IPPI_Net = IPPI_MLP_Net().cuda()
    #print(IPPI_MLP_Net())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(IPPI_Net.parameters(),lr=0.0001,weight_decay=0.001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    X_train=torch.FloatTensor(X_train).cuda()
    y_train=torch.FloatTensor( y_train).cuda()
    X_test=torch.FloatTensor(X_test).cuda()
    y_test=torch.FloatTensor( y_test).cuda()
    for epoch in range(100):
      output = IPPI_Net(X_train)
      output=torch.squeeze(output, 1)
      target =   y_train
      loss = criterion(output, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #if loss<0.15:
        #break;
      params = list(IPPI_Net.parameters())
      test_score = IPPI_Net(X_test)
      test_score=torch.squeeze(test_score, 1)
      Y_score.extend(test_score.cpu().data.numpy())
      Y_t.extend(y_test.cpu().data.numpy())
      #test_score.tolist()
      #y_test.tolist()
      auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      #Max_auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      if (auc_roc-Max_auc_roc)>0:
        Max_auc_roc=auc_roc
        print("Max auc",Max_auc_roc)
      else:
        break;
      print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())
#            print("Fauc_roc",auc_roc)
      AUC_list.append(auc_roc)
      Y_t,Y_score=[],[]
      LP.append(loss.cpu().data.numpy())
    #Testing
    test_score = IPPI_Net(X_test)
    test_score=torch.squeeze(test_score, 1)
    test_score=test_score.cpu().data.numpy()
    y_test=y_test.cpu().data.numpy()
    Z=np.append(Z,test_score);Yo=np.append(Yo,y_test);
    Auc=roc_auc_score(np.array(y_test), np.array(test_score))
    print("len of Z,Yo",len(Z),len(Yo),"Test P",t[0],"Final auc_roc",Auc)

    average_P_score=average_precision_score(y_test,test_score)
    print(testPos[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",len(testPos),"\t",len(testNeg),"\t",round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
  ######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
#    ######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
plt.figure()
plt.plot(LP)
plt.plot( AUC_list)
plt.grid()

In [ ]:
path='/content/PPI-Inhibitors/'
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
result_list=[];target_list=[];Pfeatures_dict={};target_list=[]
print("Total samples",len(ProteinData_dict))
for p in ProteinData_dict:
  #print("sample",p)
  result,PFeatures=model(ProteinData_dict[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  Pfeatures_dict[p]=PFeatures
  #print(PFeatures.shape)
###############
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel as kernel
from sklearn.svm import LinearSVC,SVC
import pandas as pd
import random
from sklearn.metrics import auc,precision_recall_curve
CC=comp.keys()
CC=list(CC);KK=[]
[KK.append(k[0].split('_')[0]) for k in CC]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set (KK)))
###########
Loss=[];AUC_list=[];LP=[];Y_t=[];Y_score=[];Z=[];Yo=[];
Complexnames=list (complex_ligand_dict.keys())
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    trainPos,trainNeg=GenerateNegativeBinderonly(trainindex,CC,complex_ligand_dict,New_binders_dict)
    train=np.vstack((trainPos,trainNeg))
    #All examples passed for negative examples generation
#        posexamples=testindex
    testPos,testNeg=GenerateNegativeBinderonly(testindex,CC,complex_ligand_dict,New_binders_dict)
#        AlltestExamples.extend(testPos);AlltestExamples.extend(testNeg);
    ######
    Ctr=[];Ptr=[];y_train=[]
    for t in trainPos:
        if getFP(Ldict[t[1]])is not None:
            Ctr.append(getFP(Ldict[t[1]]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(1)
    #####
    for t in trainNeg:
        if getFP(t[1]) is not None:
            Ctr.append(getFP(t[1]));Ptr.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_train.append(-1);
    #######
    PC_train_Features=np.hstack((Ptr,Ctr))
#        ######

    Ctt=[];Ptt=[];y_test=[]
    for t in testPos:
        if getFP(Ldict[t[1]])is not None:
            Ctt.append(getFP(Ldict[t[1]]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(1)
    ####
    for t in testNeg:
#             if t[0]!='1TUE_A_2_B':
        if getFP(t[1]) is not None:
            Ctt.append(getFP(t[1]));Ptt.append( Pfeatures_dict[t[0]][0].cpu().data.numpy());y_test.append(-1);
    #######
    PC_test_Features=np.hstack((Ptt,Ctt))
    X_train,X_test=PC_train_Features,PC_test_Features
    ######################################3
    Max_auc_roc=0.0
    IPPI_Net = IPPI_MLP_Net().cuda()
    #print(IPPI_MLP_Net())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(IPPI_Net.parameters(),lr=0.0001,weight_decay=0.001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    X_train=torch.FloatTensor(X_train).cuda()
    y_train=torch.FloatTensor( y_train).cuda()
    X_test=torch.FloatTensor(X_test).cuda()
    y_test=torch.FloatTensor( y_test).cuda()
    for epoch in range(100):
      output = IPPI_Net(X_train)
      output=torch.squeeze(output, 1)
      target =   y_train
      loss = criterion(output, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      params = list(IPPI_Net.parameters())
      test_score = IPPI_Net(X_test)
      test_score=torch.squeeze(test_score, 1)
      Y_score.extend(test_score.cpu().data.numpy())
      Y_t.extend(y_test.cpu().data.numpy())
      auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      #Max_auc_roc=roc_auc_score(np.array(Y_t), np.array(Y_score))
      if (auc_roc-Max_auc_roc)>-0.01:
        Max_auc_roc=auc_roc
        print("Max auc",Max_auc_roc)
      else:
        break;
      print("Epoch",epoch,"auc_roc",auc_roc,"loss",loss.cpu().data.numpy())
      AUC_list.append(auc_roc)
      Y_t,Y_score=[],[]
      LP.append(loss.cpu().data.numpy())
    #Testing
    test_score = IPPI_Net(X_test)
    test_score=torch.squeeze(test_score, 1)
    test_score=test_score.cpu().data.numpy()
    y_test=y_test.cpu().data.numpy()
    Z=np.append(Z,test_score);Yo=np.append(Yo,y_test);
    Auc=roc_auc_score(np.array(y_test), np.array(test_score))
    print("len of Z,Yo",len(Z),len(Yo),"Test P",t[0],"Final auc_roc",Auc)

    average_P_score=average_precision_score(y_test,test_score)
    print(testPos[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3) ,"\t",len(testPos),"\t",len(testNeg),"\t",round (len(testNeg )/len(testPos),1))
    AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
#####
fpr, tpr, thresholds = roc_curve(Yo, Z)#(Y_t, Y_score)
Auc = roc_auc_score(Yo, Z)
Auc=(Auc).round(2)
# calculate precision-recall curve
Z=np.array(Z);Yo=np.array(Yo);
#Y_t=np.array(Y_t);Y_score=np.array(Y_score)
precision, recall, thresholds = precision_recall_curve(Yo, Z)
aucpr=average_precision_score (Yo, Z)
  ######
fig = plt.figure()
plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
plt.title('AUC-PR');plt.xlabel('recall');plt.ylabel('precision');plt.grid();plt.legend();plt.show();
fig .savefig(path+"AUC-PR PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
aucpr=(aucpr).round(2)
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
#    ######
fig = plt.figure()
plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
fig .savefig(path+"PPI Inhibitors MLP.pdf", bbox_inches='tight')
###
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))
plt.figure()
plt.plot(LP)
plt.plot( AUC_list)
plt.grid()

In [ ]:
PdBloc='/content/PPI-Inhibitors/Data/Pdb'
pos_Pid,pos_smiles,pos_label=readFile('/content/PPI-Inhibitors/Data/2p2iInhibitorsSMILES.txt')
import pandas as pd
fields=['(Complexname)','Binders SMILES']
df=pd.read_csv('/content/PPI-Inhibitors/Data/BindersWithComplexname.csv', skipinitialspace=True, usecols=fields)
# See the keys
#print(df.keys())
neg_Pidname,neg_smiles=df[df.keys()[0]].values,df[df.keys()[1]].values
neg_Pid=[n.split('_')[0] for n in neg_Pidname]
####
neg_label=-1.0*(np.ones(len(neg_smiles)))
#####
compound_iso_smiles=np.append(pos_smiles,neg_smiles)
smile_graph = {}
for smile in compound_iso_smiles:
    g = smile_to_graph(smile)
    smile_graph[smile] = g
########
smiles_all,Pid_all,labels_all=np.array(np.append(pos_smiles,neg_smiles)),np.array(np.append(pos_Pid,neg_Pid)),np.array(np.append(pos_label,neg_label))
Processed_data=PrepairDataset.process(smiles_all,Pid_all,labels_all,smile_graph)

In [ ]:
print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))

In [ ]:
print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")

In [ ]:
#GNN_data,GCN_data,label=Processed_data[0]
GNN_data_all,GCN_data_all,labels=[],[],[]
for i in range(len(Processed_data)):
  GNN_data,GCN_data,label=Processed_data[i]
  GNN_data_all.append(GNN_data);GCN_data_all.append(GCN_data);labels.append(label);
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
result_list=[];target_list=[];mini_batch=[];target_list=[]
print("Total samples",GNN_data_all)
for p in range(0,10):#len(GNN_data_all)):
  print("sample",p)
  result,PFeatures=model(GNN_data_all[p])#[torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
  print(PFeatures.shape)
###############
modeling = GCN
model_st = modeling.__name__
cuda_name = "cuda:0"
print('cuda_name:', cuda_name)
dataset='PPI-inhibitor'
#############################################
CUDA_LAUNCH_BLOCKING = "1"
# training the model
device = torch.device(cuda_name if torch.cuda.is_available() else "cpu")
model = modeling().to(device)
####
LR = 0.0005
LOG_INTERVAL = 20
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
best_epoch = -1
NUM_EPOCHS=10
model_file_name = 'model_' + model_st + '_' + dataset +  '.model'
result_file_name = 'result_' + model_st + '_' + dataset +  '.csv'
TRAIN_BATCH_SIZE,TEST_BATCH_SIZE=10,10
TotalData=len(GCN_data_all)
train_data,test_data=GCN_data_all[0:int (TotalData/2)],GCN_data_all[int (TotalData/2):]
###################33
# make data PyTorch mini-batch processing ready
train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
#############################################
for epoch in range(NUM_EPOCHS):
    train(model, device, train_loader, optimizer, epoch+1)
    #1/0
    ###
    G,P,Sfeatures= predicting(model, device, test_loader,labels)
    print(Sfeatures.shape)

In [ ]:
G,P,Sfeatures= predicting(model, device, test_loader,labels)

In [ ]:
Sfeatures[0]

In [ ]:
# make data PyTorch Geometric ready
dataset='ppi_inhibitor'
print('preparing ', dataset + '_train.pt in pytorch format!')
train_data = PrepairDataset(root='data', dataset=dataset+'_train', xd=train_drugs, xt=train_prots, y=train_Y,smile_graph=smile_graph)

In [ ]:
Processed_data=PrepairDataset.process(smiles_all,Pid_all,labels_all,smile_graph)

In [ ]:
Processed_data=PrepairDataset.processProtein(Pid_all)#,"/content/PPI-Inhibitors/Data/Pdb/*")#labels_all,smile_graph,

In [ ]:
neg_Pid

In [ ]:
loc=glob.glob("/content/PPI-Inhibitors/Data/Pdb/*")
P1=loc[0]
print(P1)
parser = PDBParser()
with warnings.catch_warnings(record=True) as w:
  structure = parser.get_structure("", P1)
one_hot_atom=(atom1(structure))
print("\none_hot_atom.shape",one_hot_atom.shape)
#
one_hot_res=(res1(structure))
print("\none_hot_atom.shape",one_hot_res.shape)
neigh_same_res,neigh_diff_res=(neigh1(structure))
print("\neigh_same_res.shape,neigh_diff_re.shape",neigh_same_res.shape,"\n",neigh_diff_res.shape)
target=np.array(float (1.0))
#############
model=GNN()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
model=model.to(device)
#model.load_state_dict(torch.load(saved_model))
result_list=[];target_list=[];mini_batch=[];target_list=[]
#for p in range(0,len(one_hot_atom),2):
result,PFeatures=model([torch.tensor(one_hot_atom,dtype=torch.float32).to(device),torch.tensor(one_hot_res,dtype=torch.float32).to(device),torch.tensor(neigh_same_res).to(device).long(),torch.tensor(neigh_diff_res).to(device).long()])
1/0
#mini_batch.append(result)

          # explain the flatten
          #Result has a dimension of [N,1] and so we are trying to convert it to  a 1D tensor inorder to plot it
          #So we are flattening the result and also the target
result_list.append(torch.flatten(result.detach().cpu()).numpy())
target_list.append(target)
          #print(result_list)

#result_list now consist of  a number o numpy arrays like [[<array 1>],[<array 2>], ..] but we need to flatten it
# to plot it in a graph like [<array 1 contents>,<array 2 contents>,.... ]
result_list=np.array(result_list).flatten()

target_list=np.array(target_list).flatten()
plt.plot(target_list,result_list,".")
x = np.linspace(0,1,100)
y=x
plt.plot(x,y)
plt.xlabel("Actual scores")
plt.ylabel("Predicted Scores")

In [ ]:
PFeatures.shape

In [ ]:
Features=torch.squeeze(Features,1)

In [ ]:
Features.shape

In [ ]:
same_neigh

In [ ]:
torch.sum(same_neigh > -1).type(torch.float)

In [ ]:
same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)

In [ ]:
(same_neigh>-1).unsqueeze()

In [ ]:
same_neigh_features=neigh_signals_same[same_neigh]*unsqueezed_same_neigh_indicator

In [ ]:
same_neigh,diff_neigh=torch.tensor(neigh_same_res[p]).to(device).long(),torch.tensor(neigh_diff_res[p]).to(device).long()
unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(1)
unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(1)

In [ ]:
unsqueezed_same_neigh_indicator.shape

In [ ]:
result=model([torch.tensor(one_hot_atom[p],dtype=torch.float32).to(device),torch.tensor(one_hot_res[p],dtype=torch.float32).to(device),torch.tensor(neigh_same_res[p]).to(device).long(),torch.tensor(neigh_diff_res[p]).to(device).long()])

In [ ]:
Z=torch.tensor(one_hot_atom[p:p+2],dtype=torch.float32).to(device)#.squeeze(1)

In [ ]:
Z

In [ ]:
Z.shape

In [ ]:
torch.tensor(one_hot_atom[p]).to(device)

In [ ]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import Dataset
from torch.utils.data import sampler
import glob
import torch.optim as optim
import pandas as pd
#from QA.data_import import get_dataloader,data1
import matplotlib.pyplot as plt
#from QA.train import train
#from QA.test import test
seed=3
torch.manual_seed(seed)


In [ ]:
import py3Dmol
#taken from https://william-dawson.github.io/using-py3dmol.html
print("Target")
with open("QA/T0759-D1.pdb") as ifile:
    file_info_target = "".join([x for x in ifile])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(file_info_target)
view.setStyle({'model': -1}, {"cartoon": {'color': 'blue'}})
view.zoomTo()
view.show()
print("Decoy")
with open("QA/T0759TS499_1-D1.pdb") as ifile:
    file_info_decoy = "".join([x for x in ifile])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(file_info_decoy)
view.setStyle({'model': -1}, {"cartoon": {'color': 'salmon'}})
view.zoomTo()
view.show()

#The Graph Neural Network

### Input features

1. One hot encoded atom information

2. One hot encoded residue information

### GCN update equations


$$
h_{i}^{(l)}=\sigma\left( W_{\mathrm{center}}^{(l)} h_i^{(l-1)}  +\frac{1}{|\mathcal{N}_{\mathrm{same}}(i)|}\sum_{j\in \mathcal{N}_{\mathrm{same}}(i)}{W_{\mathrm{same}}^{(l)} h_j^{(l-1)}  } +\frac{1}{|\mathcal{N}_{\mathrm{other}}(i)|}\sum_{j\in \mathcal{N}_{\mathrm{other}}(i)}{W_{\mathrm{other}}^{(l)} h_j^{(l-1)} } \right)
$$


$$
h_{i}^{(1)}=\sigma\left( W_{\mathrm{center}}^{(0)} h_i^{(0)} + W_{\mathrm{residue}} r_i +\frac{1}{|\mathcal{N}_{\mathrm{same}}(i)|}\sum_{j\in \mathcal{N}_{\mathrm{same}}(i)}{W_{\mathrm{same}}^{(0)} h_j^{(0)}  } +\frac{1}{|\mathcal{N}_{\mathrm{other}}(i)|}\sum_{j\in \mathcal{N}_{\mathrm{other}}(i)}{W_{\mathrm{other}}^{(0)} h_j^{(0)} } \right)
$$

$h_i^{(0)}$: atom one-hot-encoding

$r_i$ residue one-hot-encoding

$\mathcal{N}_{\mathrm{same}}(i)$: the set of neighbouring atoms of atom $i$ that are within the same residue


$\mathcal{N}_{\mathrm{other}}(i)$: the set of neighbouring atoms of atom $i$ that are from different residues

$\sigma$: the activation function





### Steps for computing GCN output

In the following example we assume four different atom types: Carbon, Oxygen, Nitrogen, and Hydrogen.  Their one-hot-encoding is  [1,0,0,0] ,[0,1,0,0],[0,0,1,0] and [0,0,0,1], respectively.

#### Computing the node signals, $W_{\mathrm{center}}^{(0)} h^{(0)}$:


In [ ]:
one_hot_encoded_atom=torch.tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],dtype=torch.float32)
Wv = torch.randn(4, 10)
node_signals=one_hot_encoded_atom @ Wv
node_signals

#### Computing $\frac{1}{|\mathcal{N}_{i}|}\sum_{j\in \mathcal{N}_{i}} W_{\mathrm{neighbors}}^{(l)} h_j^{(l-1)} $


In [ ]:
import torch
# neigh_info:
# the indices of the closest neighbours of the atoms of a protein.
# In this example we have 4 atoms with 3 neighbours per atom.
# If no neighbour is present, the index is -1.
neigh_info=torch.tensor(
                         [
                          [1,3,-1],   #neighbours of first atom
                          [0,2,-1],   #neighbours of second atom
                          [1,3,-1],   #neighbours of third atom
                          [2,0,-1]    #neighbours of fourth atom
                         ]
                        )
atom_feat=torch.randn((4,6))
# atom_feat:
# the features of all the atoms in a protein
# Its dimensionality is [4,6] where  4 is the number of atoms and 6 is dimensionality
# of the embedding space of each atom

neigh_info.shape

In [ ]:
print(f'Atom features \n{atom_feat}')
print(f'Neighbour information \n{neigh_info}')

Computing the neighbour signals

In [ ]:
W_n = torch.randn((6,10),dtype=torch.float32)
neigh_signals=atom_feat@W_n
neigh_signals

Boolean matrix that indicates presence of a neighboring atom:

In [ ]:
neigh_indicator=(neigh_info>-1)
neigh_indicator

Reshaping it:

In [ ]:
unsqueezed_neigh_indicator=neigh_indicator.unsqueeze(2)
unsqueezed_neigh_indicator

Now its shape matches the shape of the neighbor signals:

In [ ]:
unsqueezed_neigh_indicator.shape, neigh_signals[neigh_info].shape

Next, we compute the features of neighbouring atoms:

In [ ]:
neigh_features=neigh_signals[neigh_info]*unsqueezed_neigh_indicator
print (neigh_features.shape)
neigh_features

Next step:  $\sum_{j\in \mathcal{N}_{i}} W_{\mathrm{neighbors}}^{(l)} h_j^{(l-1)} $

In [ ]:
sum_neigh=torch.sum(neigh_features, 1)
print(sum_neigh.shape)
sum_neigh

We still need to divide by the number of neighbors.  First, find the total number of  neighbouring atoms:

In [ ]:
num_neigh=torch.sum(neigh_info>-1,1)
num_neigh

In [ ]:
num_neigh = num_neigh.unsqueeze(1)
# To prevent divide by 0 error
num_neigh[num_neigh==0]=1
print(num_neigh.shape)
num_neigh

Now its shape matches the shape of `sum_neigh`:

In [ ]:
sum_neigh.shape

Finally, we can compute $\frac{1}{|\mathcal{N}_{i}|}\sum_{j\in \mathcal{N}_{i}} W_{\mathrm{neighbors}}^{(l)} h_j^{(l-1)} $

In [ ]:
final_neigh_features=(sum_neigh/num_neigh)
final_neigh_features


## GNN code

In [ ]:
class GNN_Layer(nn.Module):

    def __init__(self, filters, v_feats, trainable=True, **kwargs):

        super(GNN_Layer, self).__init__()
        self.v_feats = v_feats
        self.filters = filters

        self.trainable = trainable
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device=device
        self.Wsv = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wdr = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wsr = nn.Parameter( torch.randn(self.v_feats, self.filters, device=self.cuda_device,requires_grad=True))
        self.neighbours=10

    def forward(self, x):
        Z,same_neigh,diff_neigh = x
        print("input",x)
        print(Z.shape)
        node_signals = Z@self.Wsv
        neigh_signals_same=Z@self.Wsr
        neigh_signals_diff=Z@self.Wdr
        unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(2)
        same_neigh_features=neigh_signals_same[same_neigh]*unsqueezed_same_neigh_indicator
        diff_neigh_features=neigh_signals_diff[diff_neigh]*unsqueezed_diff_neigh_indicator
        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)

        # To prevent divide by zero error
        same_norm[same_norm==0]=1
        diff_norm[diff_norm==0]=1
        neigh_same_atoms_signal = (torch.sum(same_neigh_features, axis=1))/same_norm
        neigh_diff_atoms_signal = (torch.sum(diff_neigh_features, axis=1))/diff_norm
        final_res = torch.relu(node_signals +neigh_same_atoms_signal+neigh_diff_atoms_signal)

        return final_res,same_neigh,diff_neigh

class GNN_First_Layer(nn.Module):

    def __init__(self, filters, trainable=True, **kwargs):

        super(GNN_First_Layer, self).__init__()
        self.filters = filters

        self.trainable = trainable
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device = device
        self.Wv = nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wr = nn.Parameter( torch.randn(21,self.filters, device=self.cuda_device,requires_grad=True))
        self.Wsr= nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.Wdr= nn.Parameter( torch.randn(13, self.filters, device=self.cuda_device,requires_grad=True))
        self.neighbours=10


    def forward(self, x):

        atoms, residues,same_neigh,diff_neigh = x
        node_signals = atoms@self.Wv
        residue_signals = residues@self.Wr
        neigh_signals_same=atoms@self.Wsr
        neigh_signals_diff=atoms@self.Wdr
        unsqueezed_same_neigh_indicator=(same_neigh>-1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator=(diff_neigh>-1).unsqueeze(2)
        same_neigh_features=neigh_signals_same[same_neigh]*unsqueezed_same_neigh_indicator
        diff_neigh_features=neigh_signals_diff[diff_neigh]*unsqueezed_diff_neigh_indicator
        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)

        # To prevent divide by zero error
        same_norm[same_norm==0]=1
        diff_norm[diff_norm==0]=1
        neigh_same_atoms_signal=(torch.sum(same_neigh_features, axis=1))/same_norm
        neigh_diff_atoms_signal=(torch.sum(diff_neigh_features, axis=1))/diff_norm

        final_res = torch.relu(node_signals+residue_signals +neigh_same_atoms_signal+neigh_diff_atoms_signal)
        return final_res, same_neigh,diff_neigh


class Dense(nn.Module):
    def __init__(self, in_dims, out_dims, trainable=True, **kwargs):

        super(Dense, self).__init__()
        self.in_dims = in_dims
        self.out_dims = out_dims
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda:0" if use_cuda else "cpu")
        self.cuda_device = device

        self.W = nn.Parameter( torch.randn(self.in_dims, self.out_dims, device=self.cuda_device,requires_grad=True))

    def forward(self, x):

        Z = torch.sigmoid(torch.matmul(x, self.W))

        return Z

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GNN_First_Layer(filters=128)
        self.conv2 = GNN_Layer(v_feats=128, filters=256)
        self.conv3 = GNN_Layer(v_feats=256, filters=512)
        self.dense = Dense(in_dims=512, out_dims=1)

    def forward(self, x):
        x1=self.conv1(x)
        x2=self.conv2(x1)
        x3=self.conv3(x2)
        x=x3[0]
        x=torch.sum(x,axis=0).view(1,-1)
        x = F.normalize(x)
        x5=self.dense(x)
        x6=torch.squeeze(x5,1)

        return x6


## GNN Training

In [ ]:
data_loc=("Data/train_data/*/*")
label_loc=("Data/train_label/")
workers=0
batch_size=10
loader_info=data1(data_loc,label_loc,batch_size,workers)

train(GNN(),get_dataloader(loader_info))

## GNN Testing

In [ ]:
data_loc=("Data/test_data/*/*")
label_loc=("Data/test_label/")
workers=0
batch_size=10
temp=data1(data_loc,label_loc,batch_size,workers)
test(GNN(),get_dataloader(temp),'modelGCNL2_Global_Basic1.ckpt')

### GNN performance on the training set

In [ ]:
data_loc=("Data/train_data/*/*")
label_loc=("Data/train_label/")
workers=0
batch_size=10
temp=data1(data_loc,label_loc,batch_size,workers)
test(GNN(),get_dataloader(temp),'modelGCNL2_Global_Basic1.ckpt')